# Load dataset from NarrativeQA

We use Narrative from Question Generation and RAG Evaluation

In [ ]:
import pandas as pd
from Utils import *
data_loader = DatasetLoader()
from datasets import load_dataset

vector_DB = VectorDatabase()
embedder = Embedder()
data_processor = DataProcessor(embedder=embedder, vectordatabase=vector_DB)

In [ ]:
def extract_narrativeqa_text(split='train'):
    # 載入數據集
    dataset = load_dataset("deepmind/narrativeqa", split=split)
    
    # 用於存儲唯一文本的字典
    unique_summaries = {}
    unique_documents = {}
    
    total_summary_chars = 0
    total_document_chars = 0
    
    # 用於存儲問題和答案的列表
    questions = []
    answers = []
    
    # 從每個示例中提取文本
    for example in dataset:
        summary = example['document']['summary']['text']
        document = example['document']['text']
        metadata = example['document']['kind'] + "\\" + example['document']['summary']['title']
        
        # 只有當摘要和文檔都是唯一的時才添加到 df_doc
        if summary not in unique_summaries and document not in unique_documents:
            unique_summaries[summary] = metadata
            unique_documents[document] = metadata
            total_summary_chars += len(summary)
            total_document_chars += len(document)
        
        # 總是添加問題和答案到 df_qa
        questions.append(example['question']['text'])
        answers_text = ""
        for answer in example['answers']:
            answers_text += answer['text'] + " "
        answers.append(answers_text)
            
    # 從字典創建列表
    summaries = list(unique_summaries.keys())
    documents = list(unique_documents.keys())
    metadata = [unique_summaries[s] for s in summaries]  # 將元數據與摘要對齊
    
    # 計算平均值
    num_examples = len(summaries)
    avg_summary_chars = total_summary_chars / num_examples if num_examples > 0 else 0
    avg_document_chars = total_document_chars / num_examples if num_examples > 0 else 0
    
    # 創建 df_doc DataFrame
    df_doc = pd.DataFrame({
        'summary': summaries,
        'document': documents,
        'metadata': metadata
    })
    
    # 創建 df_qa DataFrame
    df_qa = pd.DataFrame({
        'questions': questions,
        'ground_truths': answers,
        'answers': ['' for _ in range(len(questions))],
        'context': ['' for _ in range(len(questions))]
    })
    
    print(f'唯一文檔數量: {num_examples}')
    print(f'問答對數量: {len(df_qa)}')
    print(f'平均摘要長度: {avg_summary_chars:.2f} 字符')
    print(f'平均文檔長度: {avg_document_chars:.2f} 字符')
    
    return df_doc, df_qa

In [ ]:
df_doc, df_qa = extract_narrativeqa_text(split="train")
df_doc_sample = df_doc.sample(frac=0.01)
df_qa_sample = df_qa.sample(frac=0.01)
df_doc_sample.to_parquet(".parquet/narrative_qa_doc_sample_11.parquet")
df_qa_sample.to_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
df_doc_sample

In [ ]:
# load parquet
df_doc_sample = pd.read_parquet(".parquet/narrative_qa_doc_sample_11.parquet")
df_qa_sample = pd.read_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
df_qa_sample



# Transform the dataframe into .txts

In [ ]:

import re
from bs4 import BeautifulSoup
import unicodedata

def preprocess_content(content: str) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()

    # 統一為 NFKC 正規化形式
    text = unicodedata.normalize('NFKC', text)

    # 移除 URL
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # 移除多餘的空白字符
    text = re.sub(r'[ \t]+', ' ', text).strip()

    # 移除特殊字符，但保留某些標點符號
    text = re.sub(r'[^\w\s.,!?;:()"-]', '', text)

    # 統一引號
    text = text.replace('"', '"').replace('"', '"')

    # 移除連續的標點符號
    text = re.sub(r'([.,!?;:])\1+', r'\1', text)

    # 確保句子之間有適當的空格
    text = re.sub(r'([.,!?;:])\s*', r'\1 ', text)

    return text.strip()

def write_text_to_files_by_metadata(df):
    # Ensure the dataframe has the required columns
    if not all(col in df.columns for col in ['document', 'metadata']):
        raise ValueError("Dataframe must contain 'document' and 'metadata' columns")

    # Dictionary to keep track of file handles
    file_handles = {}

    try:
        for _, row in df.iterrows():
            metadata = row['metadata']
            document = preprocess_content(row['document'])

            # Create or get file handle
            if metadata not in file_handles:
                filename = f".txt/{metadata}.txt"
                file_handles[metadata] = open(filename, 'a', encoding='utf-8')

            # Write document to file
            file_handles[metadata].write(document + "\n\n")  # Add two newlines for separation

    finally:
        # Close all file handles
        for handle in file_handles.values():
            handle.close()

    print(f"Files created: {', '.join(f'{metadata}.txt' for metadata in file_handles.keys())}")

In [ ]:
write_text_to_files_by_metadata(df_doc_sample)

# Embedder into Milvus (GPU) for txts

In [ ]:
data_processor.directory_files_process("narrative_qa_sample_gpu", ".txt/", True, True)

# Import GRAPH RAG data to neo4j

In [ ]:
knowledge_DB = KnowledgeGraphDatabase()

In [ ]:
knowledge_DB.transform_graph_rag_to_neo4j(datapath="../graph_rag_sample/output/20240906-153334/artifacts")

In [ ]:
retriever = Retriever()
retriever.global_retrieve(0)

# Test Modular RAG

In [ ]:
import pandas as pd
rag_evaluation_dataset = pd.read_parquet(".parquet/narrative_qa_qa_sample_11.parquet")
queries = rag_evaluation_dataset["questions"].tolist()[:2]
print(queries)
# print(vector_DB.list_collections())




In [ ]:
from Module import *
from Config.output_pydantic import *
workflow = WorkFlowModularHybridRAG()

results = workflow.graph.stream({
    "repeat_times": 0,
    "specific_collection": "narrative_qa_standard_gpu",
    "dataset_queries": queries,
    "all_results": [],
    "all_contexts": [[]],
})

for result in results:
    print(result)

# Test out Retriever for local retriever and global retriever

In [1]:
from Utils import *
retriever = Retriever()
knowledge_DB = KnowledgeGraphDatabase()
# knowledge_DB.create_entity_vector_index()
# knowledge_DB.create_community_weight()
result = retriever.local_retrieve(["What is the meaning of life?"])
result

Initializing sparse embedder...
Embedder initialized
Connected to Milvus at localhost:19530 with database default.
VectorDatabase initialized.
GraphDatabase initialized.
Retriever initialized
GraphDatabase initialized.


,nodes
0,"[(human_readable_id, name, description, id, de..."


In [ ]:
print(type(result[0].page_content))

NameError: name 'result' is not defined

In [ ]:
[<Record nodes=[<Node element_id='4:c66ce224-058b-46a0-b519-9d2e36be1920:3209' labels=frozenset({'__Entity__', 'Concept'}) properties={'human_readable_id': 2030, 'name': 'LIFE MEANING', 'description': "The concept of life meaning refers to Margaret's belief that one's life should have significance beyond domestic duties, highlighting her desire for personal fulfillment and purpose.", 'id': 'ad71cd13592144738b072262c694daaf', 'description_embedding': , -0.03398255631327629, -0.004914196208119392, -0.019359437748789787, 0.007300939876586199, -0.007306249346584082, -0.007879705168306828, -0.01653463765978813, -0.00033899591653607786, -0.016619592905044556, 0.028694020584225655, -0.02440372295677662, -0.012509828433394432, -0.031157756224274635, -0.01111866720020771, -0.06834741681814194, 0.02739843539893627, 0.010784151963889599, -0.005846061743795872, -0.04343395307660103, 0.01489922683686018, -0.04523927718400955, 0.042902976274490356, 0.019964750856161118, -0.008511568419635296, -0.04863753169775009, -0.03156130015850067, 0.03183740749955177, -0.007067309692502022, 0.005277915857732296, 0.05624112859368324, -0.002800905844196677, -0.0058142030611634254, -0.0237028319388628, -0.006727484054863453, -0.012754078023135662, 0.02684621885418892, -0.004003569949418306, -0.05598625913262367, 0.004911541007459164, 0.03003208339214325, 0.010146978311240673, -0.0032150682527571917, -0.015079759061336517, 0.008814224973320961, -0.04999683424830437, 0.022619638592004776, 0.00111505261156708, -0.0009789896430447698, 0.0439649298787117, 0.010969993658363819, 0.015334628522396088, -0.05054905265569687, -0.012085046619176865, -0.009138121269643307, -0.03177369013428688, -0.027717022225260735, -0.08852455765008926, -0.0010493441950529814, -0.01033813040703535, 0.06031903624534607, -0.015058520250022411, -0.0017827567644417286, -0.04706583917140961, 0.000732748827431351, 0.01928509958088398, 0.020272718742489815, 0.017554113641381264, 0.03988702595233917, -0.028396673500537872, -0.012138144113123417, 0.00210532546043396, 0.022152379155158997, 0.0037433907855302095, 0.020708119496703148, -0.014761172235012054, -0.008575285784900188, -0.02274707332253456, 0.012170002795755863, -0.01283903419971466, -0.022088661789894104, -0.004526582546532154, -0.05054905265569687, -0.021154141053557396, 0.0018862973665818572, -0.00895758904516697, 0.012329296208918095, 0.02100546658039093, -0.03718965873122215, 0.015918703749775887, 0.02018776163458824, 0.030902886763215065, 0.01136291678994894, -0.017501015216112137, 0.05110126733779907, 0.06392968446016312, -0.0459614060819149, -0.011787698604166508, 0.007316869217902422, -0.002118599833920598, 0.025019656866788864, 0.029246237128973007, -0.02495593950152397, 0.014824889600276947, -0.018000135198235512, -0.011479731649160385, -0.02659134939312935, 0.028396673500537872, -0.021939987316727638, -0.026506394147872925, -0.0003574141883291304, 0.0019062089268118143, 0.024828504770994186, -0.003560203593224287, -0.02198246493935585, -0.005676148924976587, -0.016460299491882324, -0.04266934469342232, 0.014527542516589165, -0.033345382660627365, -0.04770301282405853, 0.012541687116026878, -0.032155994325876236, 0.03417370840907097, 0.021918747574090958, -0.03413122892379761, 0.00043540148180909455, -0.042648106813430786, -0.012265578843653202, -0.045536626130342484, -0.0072690811939537525, -0.024106375873088837, 0.04965700954198837, -0.01627976819872856, 0.04625875502824783, 0.03731709346175194, -0.012074426747858524, -2.3333968783845194e-05, 0.015801887959241867, 0.027419675141572952, -0.023299289867281914, -0.007545189466327429, 0.07395453751087189, 0.03028695285320282, -0.016757648438215256, -0.017713407054543495, 0.00029834292945452034, 0.007274390663951635, -0.02710108831524849, 0.05658095329999924, -0.026060372591018677, -0.05275791883468628, 0.014994802884757519, -0.013731076382100582, -0.03001084364950657, -0.008973518386483192, -0.019805457442998886, 0.017171809449791908, -0.019433774054050446, 0.028226761147379875, -0.018754122778773308, 0.03924985229969025, 0.007794748526066542, 0.015939941629767418, -0.05131365731358528, -0.008368204347789288, 0.05195083096623421, 0.020601924508810043, -0.0023827613331377506, 0.05025170370936394, 0.022131139412522316, -0.03328166529536247, 0.006408897694200277, 0.010014234110713005, -0.028736498206853867, -0.02086741290986538, -0.016662072390317917, 0.0021265645045787096, -0.006392968352884054, -0.03786931186914444, -0.02516832947731018, -0.031858645379543304, -0.029755976051092148, -0.036042746156454086, -0.017161190509796143, 0.003844276536256075, 0.03661620244383812, -9.026616316987202e-05, 0.02805684693157673, -0.011097428388893604, 0.02223733440041542, -0.030117040500044823, -0.02268335595726967, 0.020134665071964264, 0.014187716878950596, -0.021483346819877625, -0.012966468930244446, -0.029246237128973007, -0.023341767489910126, 0.06133851408958435, -0.06082877516746521, -0.0028035608120262623, 0.053607482463121414, -0.019922273233532906, 0.0014575331006199121, 0.06690315902233124, -0.04247819259762764, 0.02417009323835373, 0.057048216462135315, -0.020686881616711617, -0.012775316834449768, 0.009939897805452347, -0.02268335595726967, -0.03540557622909546, 0.024764787405729294, -0.004659326747059822, 0.028439151123166084, -0.04808531701564789, -0.03166749328374863, -0.016640832647681236, -0.0034513534046709538, -0.011660264804959297, -0.004571715835481882, -0.026803741231560707, 0.024000179022550583, 0.06843236833810806, 0.04761805757880211, -0.05003931373357773, -0.019709881395101547, 0.03495955467224121, -0.028035609051585197, -0.036233898252248764, -0.03266573324799538, 0.030966604128479958, -0.0321984700858593, -0.021833792328834534, 0.011511590331792831, -0.04591892659664154, 0.013285055756568909, -0.015759410336613655, -0.053862351924180984, 0.004627468530088663, -0.037508245557546616, 0.025486916303634644, -0.03761444240808487, 0.019762979820370674, 0.08962898701429367, 0.022067422047257423, 0.02659134939312935, -0.0035363095812499523, 0.023150615394115448, 0.0015477992128580809, -0.0076460749842226505, 0.0032681659795343876, -0.03956843912601471, 0.02642143703997135, 0.028906410560011864, 0.031412623822689056, 0.0303081925958395, 0.03778435289859772, 0.013582402840256691, -0.013200098648667336, -0.017362961545586586, -0.008378824219107628, 0.025954177603125572, -0.006531022489070892, -0.038209136575460434, -0.010412467643618584, 0.010561141185462475, 0.004083216655999422, 0.006339870393276215, -0.0247010700404644, 0.006297392304986715, -0.00014460839156527072, 0.07013149559497833, -0.03731709346175194, 0.031412623822689056, 0.007274390663951635, -0.03544805198907852, -0.03466220572590828, -0.0025619661901146173, -0.043879974633455276, -0.01572755165398121, 0.008867323398590088, -0.002585860202088952, -0.007492091506719589, -9.532704279990867e-05, 0.013932847417891026, -0.029246237128973007, -0.040205612778663635, 0.006966423708945513, -0.013104522600769997, -0.02538072131574154, 0.03217723220586777, -0.02763206511735916, 0.027249760925769806, 0.02612408995628357, 0.031625017523765564, -0.0159611813724041, -0.007826607674360275, -0.03638257458806038, -0.024573635309934616, 0.0012026638723909855, 0.013497446663677692, -0.030138278380036354, 0.021738216280937195, 0.019922273233532906, -0.043370235711336136, -0.05522165447473526, -0.042520672082901, 0.027525870129466057, -0.04324280098080635, -0.013762935064733028, -0.012520447373390198, -0.04732070863246918, 0.013306294567883015, 0.012138144113123417, 0.021663879975676537, -0.048467621207237244, 0.010672646574676037, 0.03328166529536247, 0.003658434608951211, 0.034067511558532715, -0.027547109872102737, -0.048000358045101166, -0.05305526405572891, -0.020039089024066925, -0.005463757552206516, 0.006408897694200277, -0.009387681260704994, 0.00311418273486197, 0.023554159328341484, 0.03752948343753815, 0.012180622667074203, -0.05598625913262367, -0.008585904724895954, -0.004632778000086546, 0.025869220495224, -0.019964750856161118, 0.01969926245510578, -0.030117040500044823, -0.010991232469677925, -0.022152379155158997, -0.018616069108247757, 0.03803922235965729, 0.04275430366396904, -0.07858465611934662, 0.02565682865679264, 0.017883319407701492, -0.049487095326185226, -0.003475247183814645, 0.008739888668060303, -0.011394775472581387, 0.018361199647188187, 0.017033755779266357, -0.01489922683686018, 0.020325815305113792, -0.018605448305606842, 0.031200233846902847, 0.004510653205215931, 0.02397894114255905, -0.007178815081715584, -0.07187310606241226, 0.06809254735708237, 0.03608522564172745, 0.062400467693805695, -0.014198336750268936, 0.0013685943558812141, -0.014782411977648735, 0.05454200133681297, -0.012308056466281414, 0.023787789046764374, 0.04511184245347977, 0.07803244143724442, 0.0036451600026339293, -0.01891341619193554, -0.0344710536301136, -0.03351529687643051, 0.028694020584225655, 0.0566234327852726, -0.0028088705148547888, -0.037019748240709305, -0.006377039011567831, -0.0018663856899365783, -0.024297527968883514, -0.04014189541339874, -0.026761261746287346, 0.0036398502998054028, -0.02072935923933983, -0.026209045201539993, -0.031200233846902847, -0.03438609838485718, 0.00818767212331295, 0.024063896387815475, -0.005224817898124456, 0.028226761147379875, 0.005145171191543341, -0.007964661344885826, -0.05747299641370773, 0.03306927531957626, -0.006366419605910778, -0.011384156532585621, 0.022088661789894104, -0.0015093033434823155, -0.042860496789216995, -0.02393646165728569, -0.02516832947731018, -0.0353630967438221, 0.026017894968390465, -0.035023272037506104, 0.03398255631327629, 0.020962988957762718, 0.03608522564172745, 0.02295946329832077, -0.05912964791059494, -0.04354014992713928, -0.006573500577360392, -0.004959329031407833, -0.005644290242344141, -0.002755772788077593, 0.05543404445052147, -0.010513353161513805, 0.009621310979127884, -0.026251524686813354, 0.008782366290688515, -0.019433774054050446, 0.003071704413741827, -0.0021305468399077654, -0.006451375782489777, -0.036743637174367905, -0.006026593968272209, -0.028757737949490547, -0.03296307846903801, 0.02393646165728569, -0.02682497911155224, 0.00798059068620205, -0.05267295986413956, 0.014665596187114716, -0.06660580635070801, -0.0009298742515966296, -0.006594739854335785, 0.01965678483247757, 0.02293822541832924, 0.01270098052918911, 0.012509828433394432, -0.014187716878950596, 0.006382348947227001, -0.02419133111834526, 0.044347234070301056, -0.008877942338585854, 0.06771024316549301, 0.01770278811454773, -0.026463914662599564, 0.02177007496356964, -0.005883229896426201, -0.041862260550260544, 0.019762979820370674, -0.055094219744205475, -0.0006716864299960434, 0.014984183013439178, 0.017989516258239746, 0.01014166884124279, -0.02417009323835373, -0.0004682557191699743, 0.04022685065865517, 0.0009371751802973449, 0.005288535263389349, 0.03315423056483269, 0.004250474274158478, 0.03561796620488167, 0.02247096411883831, -0.017925798892974854, -0.02835419401526451, -0.0024982488248497248, 0.030648017302155495, 0.01392222847789526, -0.021324053406715393, -0.0023947081062942743, -0.024000179022550583, 0.01341248955577612, 0.0037301164120435715, -0.013720456510782242, 0.030223235487937927, 0.0038363118655979633, -0.011670883744955063, 0.00048119830898940563, -0.0014256744179874659, 0.011001852340996265, -0.004277023021131754, 0.039695873856544495, 0.036063987761735916, 0.02000723034143448, 0.006085001397877932, -0.017023136839270592, 0.015451443381607533, 0.011426634155213833, 0.0003008318890351802, 0.03659496456384659, 0.018902797251939774, 0.05271543934941292, -0.023256812244653702, 0.02886393293738365, -0.009977065958082676, -0.010024853982031345, 0.0030929434578865767, -0.03948348015546799, 0.04502688720822334, 0.0012497880961745977, -0.02684621885418892, 0.022046182304620743, 0.018520493060350418, -0.026294002309441566, -0.003196484176442027, 0.020325815305113792, 0.02788693457841873, 0.006408897694200277, -0.03850648179650307, -0.030924124643206596, 0.0293099544942379, -0.008283248171210289, -0.05148357152938843, -0.02298070304095745, 0.007093858439475298, 0.00695580430328846, -0.01259478460997343, 0.008102715946733952, -0.03973834961652756, -0.0016088616102933884, 0.024743547663092613, -0.03912241756916046, -0.03608522564172745, -0.00252347020432353, 0.01958244852721691, -0.02007094770669937, -0.02612408995628357, 0.02319309487938881, 0.011331058107316494, -0.030817929655313492, -0.0012524430640041828, 0.016778886318206787, 0.04033304378390312, -0.012063806876540184, -0.0011203624308109283, -0.01219124160706997, -0.0010427068918943405, -0.010460255667567253, 0.01969926245510578, -0.00360268191434443, 0.019062090665102005, -0.020389532670378685, -0.019985990598797798, -0.0017110747285187244, -0.015079759061336517, -0.025232046842575073, -0.021685117855668068, 0.016269149258732796, 0.016991278156638145, -0.015472682192921638, -0.019784219563007355, 0.0247010700404644, -0.0046009193174541, 0.004200031515210867, 0.00544251874089241, -0.00027494673850014806, -0.00793280266225338, 0.005065524484962225, 0.014421346597373486, -0.028502868488430977, -0.003923923242837191, 0.006637217942625284, 0.011883274652063847, 0.027207283303141594, -0.02158954180777073, -0.06278277188539505, 0.006876157596707344, 0.020209001377224922, 0.04317908361554146, 0.0028168351855129004, 0.013794793747365475, -0.018318722024559975, -0.013231957331299782, 0.012977088801562786, 0.027738260105252266, 0.01770278811454773, -0.006111550144851208, 0.007731031160801649, -0.013062044978141785, 0.010842558927834034, 0.007603596895933151, 0.007619526237249374, -0.02030457742512226, 0.03444981575012207, -0.024000179022550583, -1.9642016923171468e-05, -0.031370148062705994, 0.0474056638777256, -0.0222585741430521, -0.003615956287831068, 0.014771792106330395, 0.020548826083540916, -0.040948979556560516, 0.02633647993206978, -0.029586061835289, -0.01292399037629366, 0.0376356802880764, 0.043879974633455276, -0.0016739063430577517, -0.013147001154720783, -0.02544443868100643, 0.019242621958255768, -0.020209001377224922, 0.023511679843068123, 0.019083328545093536, 0.04097021743655205, -0.02244972623884678, -0.006833679508417845, -0.006626598536968231, 0.017182430252432823, 0.023108137771487236, 0.019826697185635567, 0.014049663208425045, -0.010608929209411144, -0.018594829365611076, 0.010046092793345451, 0.0018106329953297973, 0.013115142472088337, -0.00021322062821127474, 0.019741740077733994, 0.016757648438215256, -0.013539924286305904, -0.006754032801836729, 0.03512946516275406, 0.02903384529054165, 0.029840931296348572, 0.00838413368910551, -0.009669099003076553, 0.04366758465766907, -0.023872746154665947, -0.006366419605910778, 0.019019611179828644, -0.020315196365118027, 0.010592999868094921, -0.023363007232546806, 0.025465678423643112, 0.02690993621945381, 0.014941704459488392, 0.027908174321055412, 0.025104612112045288, 0.03566044569015503, -0.03816665709018707, -0.01186203584074974, -0.018902797251939774, -0.022619638592004776, -0.0046407426707446575, -0.008899182081222534, 0.0448569729924202, -0.0173735823482275, -0.018509874120354652, -0.044347234070301056, 0.04324280098080635, 0.01088503748178482, -0.03614894300699234, 0.018711645156145096, 0.0004762203898280859, 0.012913371436297894, 0.02344796247780323, 0.03559672832489014, 0.005771724507212639, 0.01932757906615734, -0.03294184058904648, -0.038187894970178604, -0.008580595254898071, 0.041607391089200974, 0.024722309783101082, -0.017288625240325928, 0.03198608011007309, 0.021812552586197853, 0.015079759061336517, 0.009716887027025223, 0.020814314484596252, 0.001887624734081328, 0.006387658417224884, -0.030456865206360817, -0.01749039627611637, 0.02788693457841873, 0.01889217644929886, 0.024297527968883514, -0.009844321757555008, -0.01391160860657692, -0.005745175760239363, -0.02151520550251007, 0.032325904816389084, -0.02761082723736763, 0.001821252633817494, -0.032325904816389084, -0.007173505146056414, -0.01013104896992445, 0.021557683125138283, 0.004555786494165659, 0.027929412201046944, -0.02535948157310486, 0.0002766060642898083, -0.02177007496356964, -0.005718627013266087, -0.0025553288869559765, 0.03786931186914444, -0.022598398849368095, 0.007592977490276098, 0.018382439389824867, 0.005782344378530979, -0.024127613753080368, 0.009313344024121761, 0.024849744513630867, 0.02202494442462921, 0.002824799856171012, 0.025550633668899536, -3.903513788827695e-05, 0.0036265759263187647, 0.04566406086087227, 0.035809118300676346, -0.0037433907855302095, -0.005376146640628576, -0.02340548485517502, -0.002758427755907178, 0.0021159450989216566, -0.008654932491481304, -0.006594739854335785, 0.006100930739194155, -0.020824935287237167, -0.008989447727799416, -0.042605627328157425, -0.011723981238901615, -0.006886777468025684, 0.00446021044626832, -0.012318676337599754, -0.0052858805283904076, 0.004521272610872984, -0.0010851850965991616, 0.002899136859923601, 0.006085001397877932, -0.023617876693606377, -0.015100997872650623, 0.035086989402770996, 0.05110126733779907, -0.05131365731358528, 0.016662072390317917, -0.026060372591018677, 0.0034194947220385075, 0.0018624033546075225, 0.003000022377818823, 0.06248542293906212, -0.026017894968390465, -0.02037891373038292, -0.015026661567389965, 0.009037235751748085, -0.005861990619450808, 0.03765691816806793, 0.03495955467224121, 0.002194264205172658, -0.01562135573476553, 0.006382348947227001, -0.021855030208826065, -0.02591169811785221, 0.007959351874887943, 0.006621288601309061, -0.013476206921041012, -0.06516154855489731, 0.05526413023471832, -0.01270098052918911, 0.027504630386829376, -0.030478104948997498, 0.03056306019425392, 0.03028695285320282, -0.0041416240856051445, 0.00813988409936428, -0.006860228255391121, -0.012754078023135662, 0.025784263387322426, -0.004306226968765259, 0.00247435481287539, 0.0026588693726807833, -0.029734736308455467, -0.01171336229890585, 0.014856748282909393, 0.003873480251058936, 0.012276197783648968, -0.0019619616214185953, 0.034747164696455, 0.010470874607563019, -0.024000179022550583, -0.02682497911155224, 0.04906231537461281, 0.007093858439475298, 0.01951873116195202, -0.019200144335627556, -0.01930633932352066, 0.00788501463830471, -0.009780604392290115, -0.010380608960986137, 0.005238092504441738, 0.001841164194047451, -0.006005354691296816, 0.05636856332421303, -0.022131139412522316, 0.007125717122107744, -0.025210808962583542, 0.002145148813724518, 0.007247841916978359, -0.05025170370936394, -0.022895745933055878, 0.04404988884925842, -0.02591169811785221, -0.011819557286798954, -0.014824889600276947, 0.01719304919242859, -0.014676216058433056, 0.022874508053064346, 0.01332753337919712, 0.022895745933055878, 0.01259478460997343, -0.02735595777630806, -0.03755072504281998, 0.026081610471010208, -0.02491346187889576, -0.03370644897222519, -0.01454878132790327, 0.012647882103919983, 0.035299379378557205, -0.008739888668060303, 0.010672646574676037, -0.0327506884932518, -0.03933480754494667, 0.0033796713687479496, -0.0011442563263699412, 0.005840751808136702, -0.027292240411043167, 0.004354014992713928, 0.018488634377717972, -0.040672872215509415, 0.0011031056055799127, -0.00670624477788806, -0.003230997594073415, -0.010024853982031345, -0.026315242052078247, 0.031646255403757095, -0.007805368397384882, 0.00572924641892314, -0.023851506412029266, 0.001543816877529025, -0.02782321721315384, -0.017479777336120605, -0.027271000668406487, 0.026463914662599564, 0.026973653584718704, 0.00142169208265841, -0.024977179244160652, -0.0011734601575881243, 0.018902797251939774, -0.01042308658361435, 0.0022646186407655478, 0.01341248955577612, -0.022662116214632988, 0.015939941629767418, -0.03340910002589226, -0.010550521314144135, -0.0018730228766798973, -0.032347146421670914, -0.003764630062505603, -0.03472592309117317, 0.005453138146549463, 0.013285055756568909, -0.05441456660628319, 0.00015854653611313552, -0.00951511599123478, -0.006918636150658131, 0.03850648179650307, 0.004680566024035215, 0.0118514159694314, -0.021196618676185608, -0.01088503748178482, -0.019083328545093536, 0.01565321534872055, 0.005331013351678848, -0.014962944202125072, 0.026187807321548462, 0.00509472843259573, 0.031858645379543304, 0.009100953117012978, -0.026294002309441566, -0.016916941851377487, 0.040184371173381805, -0.03489583730697632, -0.01867978647351265, -0.009573522955179214, 0.025996655225753784, 0.00622836546972394, 0.027164805680513382, 0.026527632027864456, -0.02786569483578205, 0.019593067467212677, 0.04048172011971474, -0.008654932491481304, -0.021504586562514305, 0.01953996904194355, -0.013529305346310139, 0.043837495148181915, -0.00360268191434443, -0.04668353497982025, 0.02903384529054165, 0.013603641651570797, -0.024318765848875046, 0.015355867333710194, -0.06159338355064392, -0.024849744513630867, -0.019943512976169586, -0.033387862145900726, 0.005888539832085371, -0.03808170184493065, -0.000984299462288618, 0.005500926170498133, 0.034301143139600754, 0.05883229896426201, -0.06303764134645462, -0.038740113377571106, 0.007752270437777042, -0.028991367667913437, -0.061933208256959915, 0.02295946329832077, 0.02319309487938881, -0.0009265556000173092, -0.005354907363653183, -0.0396321564912796, -0.04430475831031799, -0.014877988025546074, -0.0034672825131565332, 0.03544805198907852, 0.014081521891057491, 0.0048849922604858875, -0.033090513199567795, -0.026187807321548462, 0.00048119830898940563, 0.017362961545586586, -0.015993040055036545, 0.0009570867987349629, 0.00838413368910551, 0.027717022225260735, 0.022662116214632988, 0.013582402840256691, -0.02854534611105919, 0.005357562098652124, 0.0031752451322972775, -0.009992995299398899, -0.006403587758541107, 0.00769386300817132, -0.01563197560608387, -0.010651406832039356, -0.02177007496356964, 0.032389622181653976, 0.02104794606566429, 0.01639658398926258, 0.02956482395529747, 0.01965678483247757, 0.0319223627448082, -0.010715124197304249, 0.024573635309934616, -0.011023091152310371, 0.022640878334641457, 0.03682859614491463, 0.0011137251276522875, -0.012552306056022644, 0.0027000203263014555, -0.010035473853349686, -0.032835643738508224, 0.020878031849861145, 0.009302724152803421, 0.011012472212314606, -0.0047522480599582195, 0.015270911157131195, -0.0026469225995242596, 0.001481426996178925, 0.03634009510278702, -0.01623729057610035, 0.031136516481637955, -0.012541687116026878, 0.017755884677171707, -0.03882506862282753, 0.000349781388649717, -0.041628628969192505, -0.017267385497689247, -0.02659134939312935, -0.010359369218349457, -0.0006902706809341908, -0.035787880420684814, 0.01918952353298664, 0.0012623987859115005, 0.03145510330796242, -0.027674544602632523, -0.03202855959534645, 0.05615617334842682, 0.017267385497689247, 0.013380630873143673, 0.007906254380941391, 0.0034858668223023415, 0.015324008651077747, 0.005368181969970465, 0.031625017523765564, 0.0023840887006372213, -0.02249220386147499, -0.02061254344880581, -0.009366441518068314, 0.015462062321603298, 0.017171809449791908, -0.02321433275938034, -0.010959373787045479, 0.0031486961524933577, -0.008750507608056068, -0.0247010700404644, 0.015122237615287304, 0.0007227930473163724, -0.004194721579551697, 0.047745492309331894, 0.013603641651570797, 0.0027371887117624283, 0.008267318829894066, -0.03389759734272957, -0.0030398457311093807, 0.02512585185468197, 0.0202620979398489, -0.02042139135301113, 0.0430728904902935, -0.015876224264502525, -0.0014933740021660924, -0.011660264804959297, 0.003031881060451269, 0.00842130184173584, -0.0017973586218431592, -0.004980568308383226, 0.002896481892094016, 0.016577115282416344, -0.0600641667842865, -0.00897882878780365, 0.014803650788962841, -0.04392245411872864, 0.029947126284241676, -0.024786027148365974, 0.022555921226739883, -0.01676826737821102, 0.04249943420290947, -0.004200031515210867, -0.017670929431915283, -0.014230195432901382, -0.013465587981045246, -0.05195083096623421, -0.011628406122326851, 0.02886393293738365, 0.03003208339214325, 0.021313434466719627, -0.0011097427923232317, 0.010624858550727367, -0.0027146220672875643, -0.00951511599123478, 0.002323026303201914, 0.0212603360414505, 0.016481539234519005, 0.021876269951462746, 0.006674386095255613, -0.003982330672442913, -0.04783044755458832, -0.04536671191453934, 0.03459848836064339, -0.007115097716450691, -0.0049301255494356155, 0.03515070676803589, 0.014771792106330395, 0.03689231351017952, 0.004061977379024029, 0.02589046023786068, -0.0040964907966554165, 0.017458537593483925, 0.03491707518696785, -0.03292059898376465, 0.03255953639745712, -0.004598264582455158, 0.020962988957762718, -0.04519679769873619, 0.0010088571580126882, 0.039207372814416885, 0.04003569856286049, 0.030966604128479958, -0.021228477358818054, -0.01674702763557434, -0.01317885983735323, 0.008070857264101505, 0.009918658062815666, 0.05203578993678093, 0.04009941592812538, -0.0005153799429535866, 0.015451443381607533, 0.027525870129466057, 0.009509805589914322, 0.01846739463508129, 0.011320439167320728, -0.02807808667421341, 0.007890325039625168, -0.006515093147754669, -0.028014369308948517, 0.04126756638288498, 0.009552284143865108, 0.0019393950933590531, 0.0025818778667598963, 0.0044416263699531555, 0.0025486915837973356, 0.018594829365611076, -0.0012424872256815434, 0.017777124419808388, -0.020601924508810043, 0.004513307940214872, -0.0008542099385522306, -0.036063987761735916, -0.0006318631349131465, 0.028502868488430977, 0.015950562432408333, 0.00382038252428174, -0.017108092084527016, 0.004292952362447977, 0.029968366026878357, -0.008277937769889832, -0.0006428145570680499, 0.004898266866803169, -0.029267476871609688, 0.025593111291527748, 0.0053628720343112946, -0.011830177158117294, -0.010184147395193577, 0.006127479486167431, -0.05059152841567993, -0.03281440585851669, 0.011384156532585621, 0.02247096411883831, 0.027908174321055412, -0.021175380796194077, -0.005405350122600794, 0.036743637174367905, 0.03767815977334976, -0.07064123451709747, 0.0036637443117797375, 0.01930633932352066, -0.0017416059272363782, 0.0037672847975045443, 0.009977065958082676, -0.0028327645268291235, -0.016577115282416344, -0.016407202929258347, -0.006238984875380993, 0.02710108831524849, 0.041671108454465866, -0.051143746823072433, 0.01063016802072525, 0.004412422422319651, -0.025040894746780396, -0.008118645288050175, 0.016619592905044556, -0.009823082014918327, 0.0026243559550493956, 0.009233697317540646, 0.0007214655634015799, 0.01221248134970665, 0.025019656866788864, -0.05101631209254265, -0.0019354127580299973, 0.014516922645270824, -0.01877536252140999, 0.014920465648174286, -0.07051379978656769, -0.0024929391220211983, 0.04147995635867119, 0.005766415037214756, 0.015345247462391853, -0.03028695285320282, -0.009026615880429745, -0.018966514617204666, 0.01914704591035843, -0.0025380721781402826, 0.012764697894454002, 0.01658773422241211, 0.013476206921041012, 0.019040850922465324, 0.05152605101466179, 0.01872226409614086, 0.006276153028011322, 0.002534089842811227, -0.0026256833225488663, 0.0032548916060477495, 0.017479777336120605, 0.007141646463423967, -0.05560395494103432, 0.024000179022550583, -0.01756473258137703, -0.028247999027371407, 0.022598398849368095, -0.0005721281631849706, 0.00671155471354723, -0.0019473597640171647, -0.0027172770351171494, -0.011978850699961185, -0.00819829199463129, -0.005755795165896416, 0.01830810122191906, -0.008092096075415611, -0.01038591843098402, -0.011893894523382187, -0.019253240898251534, -0.03693478927016258, -0.04230828210711479, -0.022810790687799454, -0.05012426897883415, -0.0014031078899279237, -0.013720456510782242, 0.008453160524368286, 0.04400740936398506, 0.0016420476604253054, 0.0008581922738812864, 0.061423469334840775, 0.0011250084498897195, 0.0012517792638391256, 0.021302813664078712, 0.00030863063875585794, 0.02251344360411167, -0.029628541320562363, -0.024297527968883514, 0.027950651943683624, -0.029713496565818787, 0.003464627778157592, 0.039462242275476456, 0.016853224486112595, 0.04046047851443291, -0.009780604392290115, -0.05352252349257469, -0.030732974410057068, 0.004587645176798105, -0.004664636682718992, 0.007561118807643652, -0.021302813664078712, 0.0002142162120435387, -0.015324008651077747, 0.011203623376786709, -0.013603641651570797, -0.03608522564172745, -0.013805413618683815, -0.007019521668553352, -0.041862260550260544, 0.02004970796406269, -0.007019521668553352, 0.03253829851746559, 0.016789507120847702, 0.014442586340010166, -0.016640832647681236, -0.04536671191453934, 0.00411242013797164, -0.01269036065787077, 0.032580774277448654, -0.01817004755139351, -0.04659857973456383, -0.02610285021364689, -0.014612498693168163, -0.048255227506160736, 0.0134337292984128, -0.02805684693157673, -0.04664105921983719, -0.007975281216204166, -0.013688597828149796, -0.010715124197304249, 0.04075782746076584, -0.04668353497982025, -0.0024730274453759193, -0.002303114626556635, -0.015515160746872425, -0.05560395494103432, -0.012573545798659325, 0.008044308051466942, -0.02088865265250206, -0.02877897582948208, 0.0021624055225402117, -0.030881647020578384, 0.012010709382593632, -0.05743052065372467, 0.0026256833225488663, -0.01895589381456375, -0.022555921226739883, 0.005267295986413956, 0.00020691526879090816, -0.038952503353357315, 0.01317885983735323, -0.00028291140915825963, -0.023108137771487236, -0.01956120878458023, -0.038230374455451965, 0.007215983234345913, -0.010364679619669914, -0.013136381283402443, 0.016184192150831223, 0.011086808517575264, -0.009971756488084793, -0.017182430252432823, -0.02809932641685009, -0.013709837570786476, 0.018382439389824867, -0.02268335595726967, 0.02196122705936432, 0.04791540279984474, -0.01406028214842081, 0.019879795610904694, -0.003995605278760195, -0.0013871785486117005, -0.011777079664170742, -0.01438948791474104, -0.01670455001294613, -0.02637895941734314, -0.006748723331838846, 0.026230284944176674, 0.04137376323342323, -0.013263816013932228, -0.022662116214632988, -0.016757648438215256, -0.018063852563500404, 0.03727461397647858, -0.0035389645490795374, 0.01786208152770996, 0.03986578434705734, 0.017182430252432823, -0.022407246753573418, 0.0019460322801023722, -0.013890369795262814, 0.008277937769889832, -0.01663021370768547, 0.006817750167101622, 0.006249604281038046, -0.0010307598859071732, -0.02854534611105919, -0.010906276293098927, 0.00868679117411375, 0.0019951476715505123, 0.0024358590599149466, 0.0007214655634015799, 0.03347281739115715, -0.02514709159731865, 0.014187716878950596, -0.002141166478395462, 0.0359153151512146, 0.035065747797489166, -0.009913348592817783, 0.026294002309441566, 0.00799121055752039, -0.014612498693168163, -0.048255227506160736, 0.016619592905044556, 0.0364675298333168, -0.012509828433394432, 0.03323918581008911, 0.0016672691563144326, -0.024488678202033043, -0.017437299713492393, -0.019465632736682892, -0.025508156046271324, -0.03498079255223274, 0.009414229542016983, -1.5068559150677174e-05, -0.041883498430252075, -0.0022048838436603546, 0.004237199667841196, -0.01563197560608387, -0.025869220495224, -0.014708074741065502, 0.021345293149352074, 0.006573500577360392, 0.04001446068286896, 0.010454945266246796, 0.002194264205172658, 0.0313064306974411, -0.04060915485024452, 0.0014907191507518291, -0.002211520913988352, -0.03126395121216774, 0.003342502983286977, 0.033579014241695404, -0.02090989053249359, 0.009493876248598099, -0.009159361012279987, 0.008543427102267742, 0.011246101930737495, 0.012977088801562786, 0.0065575712360441685, -0.0011442563263699412, -0.008325725793838501, -0.00043540148180909455, 0.008633692748844624, -0.006116860080510378, 0.009042545221745968, 0.033833883702754974, 0.03840028867125511, 0.009610691107809544, 0.031157756224274635, -0.016736408695578575, 0.013444348238408566, 0.015239052474498749, 0.009573522955179214, 0.0011449201265349984, -0.00034347601467743516, -0.007715102285146713, -0.009525734931230545, 0.0024663901422172785, 0.03226218745112419, 0.003464627778157592, -0.016598355025053024, -0.03005332313477993, -0.029968366026878357, 0.017267385497689247, -0.01267974078655243, 0.052375614643096924, 0.007099168375134468, 0.008081476204097271, -0.03803922235965729, -0.01024786476045847, 0.019221382215619087, -0.019858555868268013, 0.0015185954980552197, -0.021812552586197853, 0.018902797251939774, -0.004813310224562883, 0.0011973540531471372, -0.019359437748789787, -0.030690494924783707, 0.023554159328341484, 0.0307542122900486, -0.018998373299837112, -0.004539857152849436, 0.003926577977836132, 0.02807808667421341, -0.024297527968883514, -0.01565321534872055, -0.0015331973554566503, 0.04681096971035004, -0.0307542122900486, -0.006647837348282337, 0.005840751808136702, -0.010109810158610344, -0.031115276739001274, -0.030117040500044823, 0.011288580484688282, -0.023023180663585663, 0.030605539679527283, -0.028396673500537872, 0.02344796247780323, 0.012817795388400555, -0.037019748240709305, -0.009520425461232662, -0.04048172011971474, 0.030350670218467712, 0.05059152841567993, -0.012424872256815434, -0.004059322644025087, -0.025763025507330894, 0.023108137771487236, 0.03323918581008911, -0.037040986120700836, 0.014262054115533829, -0.01863730698823929, 0.00533366808667779, -0.022279813885688782, 0.016948798671364784, -0.005888539832085371, -0.011500971391797066, 0.015079759061336517, 0.008644312620162964, 0.033048033714294434, -0.010805390775203705, 0.012244339101016521, -0.017150571569800377, 0.008012449368834496, 0.007056690286844969, 0.05076144263148308, -0.013093903660774231, 0.02123909629881382]}>, <Node element_id='4:c66ce224-058b-46a0-b519-9d2e36be1920:1535' labels=frozenset({'__Entity__', 'Concept'}) properties={'human_readable_id': 356, 'name': 'LIFE', 'description': 'The entity "LIFE" is depicted as a multifaceted concept that encapsulates the experiences and challenges faced by various characters, particularly Dirk, Dolcy, and Dorothy. For Dirk, life is characterized by a tension between his mother\'s expectations and his personal aspirations, highlighting the struggle for identity and fulfillment. In the context of Dolcy, life is associated with themes of sacrifice and emotional turmoil, reflecting the narrator\'s complex feelings regarding her actions. Furthermore, life is portrayed as confusing and complicated, mirroring the characters\' internal conflicts and the difficult decisions they must navigate. Dorothy\'s love is intricately linked to her existence, suggesting that her sense of self is deeply intertwined with her emotional connections. Overall, life emerges as a rich tapestry of struggles, emotions, and relationships that shape the characters\' journeys.', 'id': 'd4602d4a27b34358baa86814a3836d68', 'description_embedding': [0.047583699226379395, 0.029947783797979355, -0.025813965126872063, 0.04945223778486252, -0.02805365063250065, 0.00338832288980484, 0.06557796895503998, 0.038266610354185104, 0.03790826350450516, 0.025609193369746208, -0.00389065220952034, -0.03196989744901657, -0.03824101388454437, -0.05155114084482193, -0.017149586230516434, 0.01647128164768219, -0.006751049309968948, 0.02792566828429699, -0.004610551055520773, 0.04256680607795715, 0.038983311504125595, -0.0016477679600939155, -0.007806900888681412, 0.0286167711019516, 0.013105355203151703, -0.011550373397767544, 0.03194430097937584, 0.054110780358314514, 0.03514385223388672, -0.016458483412861824, -0.026645848527550697, -0.014641138724982738, 0.0656803548336029, 0.003708277828991413, 0.004159414209425449, 0.019786013290286064, -0.009560253471136093, 0.012241476215422153, 0.03872734680771828, -0.019146103411912918, -0.05574895069003105, -0.009227501228451729, -0.007320569362491369, 0.030638886615633965, -0.017904678359627724, 0.006296713370829821, 0.0058999694883823395, 0.05902528762817383, 0.02023395150899887, 0.05267738178372383, -0.04602231830358505, 0.02193611115217209, 0.04289956018328667, 0.025749973952770233, -0.07151632755994797, -0.007269376423209906, -0.023958226665854454, -0.018518991768360138, -0.010500921867787838, 0.0011190423974767327, 0.007883690297603607, -0.006597470957785845, -0.015370636247098446, 0.025801166892051697, -0.021884918212890625, -0.007166991010308266, -0.039827991276979446, 0.009137913584709167, -0.033556874841451645, 0.0463806688785553, 0.014359578490257263, 0.024713320657610893, 0.008235640823841095, -0.01066729798913002, -0.027695300057530403, -0.002252482809126377, 0.009489864110946655, 0.0007462949142791331, -0.011723149567842484, -0.044947270303964615, 0.029435856267809868, 0.0035866950638592243, -0.0001879735355032608, 0.038420189172029495, 0.012241476215422153, -0.029103102162480354, -0.03944404795765877, 0.014193201437592506, -0.018429405987262726, -0.01556260883808136, 0.04553598910570145, -0.020285144448280334, -0.028565578162670135, 0.015946554020047188, 0.037601105868816376, -0.006559076253324747, 0.057796660810709, -0.011064042337238789, -0.022294461727142334, 0.01014897134155035, 0.011332804337143898, 0.026569059118628502, 0.0007594930357299745, 0.021769734099507332, 0.04881232604384422, 0.0006835037493146956, -0.015447424724698067, -0.013118152506649494, -0.023548685014247894, 0.005474429111927748, -0.1069929376244545, 0.037601105868816376, 0.01254863291978836, 0.05958840996026993, -0.01123041845858097, 0.010692894458770752, -0.022729599848389626, 0.032609809190034866, 0.02766970358788967, 0.0023436699993908405, -0.021027440205216408, 0.02971741557121277, -0.016829630360007286, 0.003151556244120002, 0.026927407830953598, 0.03071567602455616, 0.04359066113829613, -0.00399303762242198, -0.02641548030078411, -0.012887785211205482, -0.009432272054255009, -0.013796457089483738, -0.0509624257683754, -0.05224224552512169, -0.0024748514406383038, -0.015831371769309044, -0.043155524879693985, 0.002937186509370804, 0.022486433386802673, 0.019811609759926796, -0.002126100705936551, -0.034683115780353546, 0.018186239525675774, -0.0040442305617034435, 0.08503122627735138, 0.07402477413415909, 0.030101362615823746, 0.04489607736468315, 0.016548069193959236, -0.00878596305847168, 0.00972663052380085, 0.0165992621332407, -0.021296201273798943, 0.006536679808050394, 0.012651018798351288, -0.059281252324581146, -0.0034747107420116663, 0.013514896854758263, -0.039648815989494324, -0.005912767257541418, 0.006507883779704571, -0.011633561924099922, -0.011979113332927227, 0.00930428970605135, 0.000264962698565796, -0.005615209229290485, -0.02089945785701275, 0.001650967518799007, -0.033684857189655304, -0.018518991768360138, -0.03752431645989418, 0.023100746795535088, -0.046764615923166275, -0.0047577302902936935, 0.014167604967951775, -0.05626088008284569, 0.03404320776462555, -0.021347394213080406, -0.04965700954198837, -0.03112521767616272, -0.00574959022924304, -0.02971741557121277, -0.015575407072901726, -0.030638886615633965, -0.03309613838791847, 0.01964523456990719, -0.01737995259463787, 0.0052824560552835464, -0.00031855516135692596, -0.0055064246989786625, 0.01024495717138052, 0.029282277449965477, -0.0020221152808517218, -0.012593426741659641, 0.011953516863286495, 0.0366540402173996, 0.05365004763007164, 0.009214702993631363, -0.01709839329123497, 0.0325586162507534, 0.016855226829648018, 0.03099723532795906, 0.03962321951985359, -0.043283507227897644, -0.039930377155542374, 0.020963449031114578, -0.0006355105433613062, -0.013591686263680458, -0.007166991010308266, -0.01917169988155365, 0.021820927038788795, -0.036474864929914474, -0.0013574088225141168, -0.017802294343709946, 0.007947680540382862, 0.01757192611694336, 0.01507627684623003, -0.01348930038511753, -0.008229241706430912, 0.007339766714721918, 0.0185701847076416, 0.04210606962442398, 0.04748131334781647, 0.014001228846609592, -0.010500921867787838, 0.007026210892945528, -0.004162613768130541, -0.06030510738492012, -0.01460274402052164, -0.01759752258658409, 0.010161769576370716, -0.001481391373090446, -0.03557899221777916, -0.008913944475352764, -0.002742013894021511, -0.03923927620053291, -0.018096651881933212, -0.002284478396177292, 0.019773215055465698, 0.02736254781484604, -0.04696938768029213, 0.023894235491752625, 0.029973380267620087, 0.03534862399101257, -0.03212347626686096, -0.008402016945183277, 0.004789725411683321, -0.009400276467204094, -0.04778847098350525, -0.029103102162480354, 0.018403809517621994, 0.002780408598482609, 0.031739529222249985, -0.006475888192653656, 0.020105969160795212, 0.001964523456990719, 0.019913995638489723, -0.015767380595207214, 0.06588512659072876, 0.020131565630435944, 0.05615849420428276, 0.0234718956053257, -0.012439848855137825, 0.01024495717138052, -0.0021452980581671, -0.04269478842616081, 0.004412178881466389, 0.012638220563530922, -0.0060791438445448875, 0.020745879039168358, -0.023638272657990456, -0.03795945644378662, -0.014014027081429958, 0.02079707197844982, -0.06532200425863266, -0.06404218077659607, -0.00889474805444479, 0.011505579575896263, 0.0572335422039032, 0.04776287451386452, 0.0194532610476017, -0.004383382853120565, 0.027208968997001648, -0.04891471192240715, -0.04254120960831642, 0.005266458261758089, 0.009400276467204094, -0.02513566054403782, -0.006197527516633272, 0.00573359290137887, -0.01731596142053604, -0.017674311995506287, -0.030485307797789574, 0.0002425658458378166, 0.021372990682721138, -0.043974608182907104, -0.03266100212931633, -0.05999795347452164, 0.032891370356082916, 0.04144056513905525, 0.032456230372190475, 0.0027980059385299683, 0.004450573585927486, 0.053701240569353104, -0.018710965290665627, 0.028284016996622086, 0.02848878875374794, -0.034683115780353546, 0.028667964041233063, 0.006421495694667101, 0.026799427345395088, 0.00851720105856657, 0.07074844092130661, 0.03373605012893677, -0.01319494191557169, -0.01194071862846613, 0.03683321550488472, 0.027029793709516525, 0.006706255953758955, -0.012247875332832336, -0.022819187492132187, -0.005685599520802498, 0.010680096223950386, 0.0008238839800469577, 0.0194532610476017, -0.004047430120408535, -0.02654346264898777, 0.010571311227977276, -0.04881232604384422, -0.026927407830953598, -0.024956485256552696, -0.018518991768360138, -0.03957202658057213, 0.0061591328121721745, -0.0058327787555754185, -0.05452032387256622, 0.03324971720576286, -0.018646974116563797, 0.029845397919416428, 0.001147838425822556, -0.03324971720576286, 0.0073845600709319115, -0.05574895069003105, -0.005880772136151791, -0.05436674505472183, 0.00730137201026082, 0.01625371165573597, -0.057335928082466125, 0.009905804879963398, 0.02710658311843872, 0.0007282974547706544, -0.016445685178041458, -0.028693560510873795, -0.010206562466919422, -0.06916145980358124, -0.0042969947680830956, -0.0031851516105234623, -0.031329989433288574, -0.0031963500659912825, 0.025967543944716454, -0.03212347626686096, -0.06772806495428085, -0.015946554020047188, 0.07238660752773285, -0.06235282123088837, -0.014897102490067482, 0.026287497952580452, -0.02416299842298031, -0.015217057429254055, 0.005727193783968687, 0.05111600458621979, -0.0343247689306736, 0.004261800087988377, 0.055646564811468124, 0.011441589333117008, 0.07581652700901031, -0.03166274353861809, -0.014564349316060543, -0.03839459270238876, -0.017277566716074944, -0.03191870450973511, 0.009470666758716106, 0.031355585902929306, 0.013809255324304104, 0.012087898328900337, 0.045356813818216324, -0.028667964041233063, -0.03893211856484413, -0.007454950362443924, -0.023228729143738747, 0.034529536962509155, -0.04791645333170891, 0.021795330569148064, -0.041466161608695984, 0.021449780091643333, -0.0320722833275795, 0.007275775540620089, 0.08800040930509567, 0.041773319244384766, -0.06178969889879227, 0.030459711328148842, 0.030408518388867378, -0.015460222959518433, -0.008881949819624424, -0.026159517467021942, 0.014948295429348946, 0.030920445919036865, -0.009688235819339752, 0.002785207936540246, -0.01960683986544609, 0.014756321907043457, -0.010526517406105995, 0.016560867428779602, -0.0004057428741361946, -0.0077109141275286674, -0.04617589712142944, 0.027720896527171135, 0.0020701084285974503, 0.04113340750336647, 0.06383740901947021, 0.028744753450155258, -0.033121734857559204, 0.01666325330734253, -0.05283096060156822, 0.026876216754317284, 0.06363263726234436, 0.039674412459135056, 0.02159056067466736, 0.0037754683289676905, 0.008434012532234192, 0.0063159107230603695, 0.0475069098174572, 0.0071989865973591805, -0.0234335009008646, 0.009125115349888802, -0.029256680980324745, 0.013009368441998959, -0.02111702598631382, -0.04512644559144974, -0.06665301322937012, -0.010372939519584179, 0.01891573704779148, 0.005644005257636309, 0.011684754863381386, -0.029538240283727646, 0.010808078572154045, 0.047430120408535004, -0.035092659294605255, -0.025903552770614624, -0.0025900353211909533, -0.0012078299187123775, -0.0027164174243807793, 0.011320006102323532, 0.020873861387372017, -0.024073410779237747, -0.006559076253324747, 0.017559127882122993, -0.027720896527171135, 0.017623119056224823, -0.007435753010213375, -0.042464420199394226, -0.01256783027201891, -0.0008278834284283221, -0.014551551081240177, -0.0007091001607477665, 0.010641701519489288, 0.06660182029008865, -0.07771065831184387, -0.045356813818216324, 0.010788881219923496, -0.022576021030545235, -0.006923825014382601, 0.0033627264201641083, -0.0007682918221689761, 0.00520886667072773, 0.002102104015648365, -0.036602847278118134, 0.021193815395236015, -0.00033035347587428987, 0.030690079554915428, 0.025084467604756355, 0.05045049637556076, -0.041619740426540375, 0.002988379215821624, -0.029231084510684013, 0.006482287310063839, -0.002866796450689435, -0.01885174587368965, 0.020438721403479576, 0.002153296722099185, -0.01867257058620453, -0.05124398320913315, 0.028565578162670135, -0.013207740150392056, -0.010641701519489288, -0.002484450116753578, -0.002892392687499523, 0.004479369148612022, 0.025020476430654526, -0.019248489290475845, 0.021168218925595284, 0.051755912601947784, 0.024636531248688698, 0.01640729047358036, 0.025993140414357185, -0.011595167219638824, 0.04069827124476433, -0.020310740917921066, -0.043001946061849594, 0.009278693236410618, -0.04794204980134964, 0.026364287361502647, 0.042873963713645935, 0.03450394049286842, 0.004149815533310175, -0.04249001666903496, -0.02667144499719143, 0.0005903168930672109, -0.021782532334327698, -0.001204630360007286, -0.01149918045848608, -0.035758163779973984, 0.013284529559314251, 0.027848878875374794, 0.02513566054403782, -0.02325432561337948, 0.002929187612608075, 0.0032987354788929224, 0.0038042643573135138, 0.013450905680656433, -0.0022924772929400206, -0.015025084838271141, 0.01679123565554619, 0.01466673519462347, -0.015869764611124992, 0.048146821558475494, 0.007019811775535345, -0.04072386771440506, 0.04568956792354584, -0.036628443747758865, 4.6918394218664616e-05, 0.010564912110567093, 0.04233643785119057, 0.03836899623274803, 0.02818163111805916, 0.033966418355703354, -0.04072386771440506, 0.003865055739879608, -0.002398062264546752, 0.0062487199902534485, 0.0795024037361145, 0.019568445160984993, 0.027439337223768234, -0.0050104944966733456, 0.02864236757159233, 0.015895361080765724, -0.0014006027486175299, 0.0055736154317855835, -0.01772550493478775, 0.0183270201086998, -0.031074024736881256, -0.029256680980324745, 0.0059895566664636135, 0.007237380836158991, -0.01424439437687397, 0.011659158393740654, 0.02321593090891838, 0.00876676570624113, 0.0007330967928282917, -0.01735435612499714, -0.01010417751967907, -0.007026210892945528, 0.009125115349888802, -0.018800552934408188, 0.0059895566664636135, -0.007922085002064705, -0.010129773989319801, -0.010187366046011448, -0.011083239689469337, -0.032430633902549744, -0.019286883994936943, -0.059690795838832855, -0.030357325449585915, -0.021859321743249893, -0.0257243774831295, 0.02284478396177292, -0.016740042716264725, -0.009438671171665192, -0.008280433714389801, 0.01681683212518692, -0.011748746037483215, 0.014705129899084568, -0.005058487877249718, 0.03442715108394623, 0.015856968238949776, -0.00983541551977396, 0.050783250480890274, 0.0008034868515096605, -0.012491040863096714, -0.033275313675403595, -0.038138631731271744, 0.010801679454743862, -0.020105969160795212, -0.027874475345015526, -0.010142572224140167, -0.02537882700562477, -0.007634125184267759, 0.0017485538264736533, 0.031202007085084915, 0.012055902741849422, -0.0002499647962395102, -0.020733080804347992, -0.007934882305562496, -0.02152656950056553, -0.0015485819894820452, 0.007422954775393009, -0.009912203997373581, 0.008504402823746204, 0.05221664905548096, -0.03918808326125145, -0.04909388720989227, -0.03294256329536438, 0.008274034596979618, 0.010660898871719837, 0.02438056655228138, -0.029000716283917427, -0.04530562087893486, 0.012983771972358227, -0.0052472613751888275, 0.010801679454743862, -0.032993756234645844, 0.013758063316345215, -0.0031355584505945444, 2.1996902432874776e-05, 0.02557080052793026, 0.020131565630435944, -0.0008702774648554623, 0.0081588514149189, -0.01647128164768219, -0.016266509890556335, 0.021692946553230286, 0.029282277449965477, 0.03570697084069252, -0.017610320821404457, 0.013514896854758263, -0.01819903776049614, -0.0018045458709821105, 0.0024492552038282156, 0.014474761672317982, -0.025647588074207306, 0.02669704146683216, -0.009355482645332813, 0.030766867101192474, -0.004837718792259693, 0.021232210099697113, 0.018314221873879433, -0.039955973625183105, 0.003237944096326828, 0.006770246662199497, 0.03281458094716072, -0.02168014831840992, -0.0041402168571949005, 0.027823282405734062, 0.008702774532139301, 0.030766867101192474, 0.0020061174873262644, 0.013297327794134617, -0.031739529222249985, -0.013073359616100788, 0.009784222580492496, 0.03268659859895706, 0.02406061254441738, 0.03235384449362755, 0.03212347626686096, -0.010404935106635094, -0.0008446810534223914, 0.003105162875726819, -0.02416299842298031, -0.02457254007458687, 0.0037498720921576023, -0.029538240283727646, 0.009016330353915691, 0.005074485670775175, -0.0036250895354896784, 0.046201493591070175, -0.023292720317840576, 0.059537217020988464, -0.00945786852389574, -0.004709736909717321, 0.012241476215422153, -0.04881232604384422, 0.01835261657834053, -0.009701034054160118, 0.0051192790269851685, 0.02751612663269043, -0.017891880124807358, 0.02933347038924694, 0.0039546433836221695, 0.014461963437497616, 0.0029323871713131666, 0.0023052755277603865, -0.004946503322571516, -0.03957202658057213, -0.033121734857559204, -0.04126138985157013, -0.07294972985982895, 0.020669089630246162, 0.006303112488240004, 0.04126138985157013, -0.05157673731446266, 0.022256067022681236, -0.0027836081571877003, 0.013310126028954983, 0.03491348400712013, -0.03360806778073311, 0.02989659085869789, -0.05380362272262573, 0.022294461727142334, -0.01804545894265175, 0.04366745054721832, 0.002252482809126377, 0.03322412073612213, -0.043411485850811005, -0.035911742597818375, -0.030178150162100792, 0.0050232927314937115, 0.005055288318544626, -0.015485819429159164, -0.0051736715249717236, 0.012516637332737446, 0.016445685178041458, 0.03859936445951462, 0.006981417071074247, -0.012619023211300373, -0.017866283655166626, -0.028156036511063576, -0.0337616465985775, 0.025609193369746208, 0.004005835857242346, 0.039264872670173645, -0.039111293852329254, -0.015396231785416603, -0.03015255369246006, 0.009867411106824875, 0.010270553641021252, -0.008779563941061497, -0.0010582510149106383, -0.013694072142243385, 0.006101540755480528, -0.003797865239903331, -0.002015716163441539, 0.010539315640926361, -0.0061463345773518085, -0.03754991292953491, 0.000222768634557724, -0.017021603882312775, -0.023676665499806404, 0.02252482809126377, 0.0223072599619627, -0.030331728979945183, 0.04323231428861618, 0.03580935671925545, -0.005695198196917772, -0.012228677980601788, -0.02127060480415821, 0.013271731324493885, -0.0046969386748969555, -0.0024076609406620264, 0.04643186181783676, -0.024623733013868332, -0.025212449952960014, 0.02139858715236187, 0.030920445919036865, -0.011364799924194813, 0.006898228544741869, -0.05743831396102905, -0.008779563941061497, 0.036782022565603256, 0.006386301014572382, -0.017085595056414604, -0.0026412280276417732, -0.022076891735196114, 0.004588154144585133, -0.03332650661468506, 0.0061495341360569, -0.00542003707960248, -0.015575407072901726, 0.013322924263775349, 0.008210044354200363, -0.011761544272303581, -0.021910514682531357, 0.010187366046011448, 0.00489531084895134, 0.007640524301677942, -0.01451315637677908, 0.03156035766005516, 0.008709173649549484, -0.09808538854122162, 0.021436981856822968, -0.051499947905540466, -0.019274085760116577, 0.025558002293109894, -0.005480828229337931, 0.05283096060156822, -0.004892111290246248, -0.005368844140321016, -0.022409643977880478, 0.02083546668291092, -0.007410156540572643, 0.018096651881933212, -0.016023343428969383, 0.021372990682721138, -0.010424132458865643, 0.011115235276520252, 0.002260481705889106, -0.0217441376298666, -0.011787140741944313, 0.0003687480930238962, 0.0037434729747474194, -0.02221767231822014, 0.06030510738492012, 0.026313094422221184, 0.011038445867598057, -0.028693560510873795, -0.0383433997631073, 0.01958124339580536, -0.024175796657800674, 0.021372990682721138, -0.008913944475352764, -0.027541721239686012, 0.010955257341265678, -0.0037370738573372364, 0.012996570207178593, 0.023267123848199844, -0.004239403177052736, -0.016586463898420334, 0.017802294343709946, 0.022819187492132187, 0.021296201273798943, 0.028028054162859917, 0.005493626464158297, 0.0027276158798485994, -0.02613392099738121, -0.06322310119867325, 0.034683115780353546, -0.010481724515557289, -0.004082625266164541, -0.037447527050971985, 0.0019165301928296685, 0.05518582835793495, -0.05098802223801613, -0.036321286112070084, 0.0234718956053257, -0.003181952051818371, 0.001405402086675167, -0.035655777901411057, -0.02987099438905716, 0.012842991389334202, -0.008024469949305058, 0.0023932629264891148, 0.02086106315255165, -0.06813760846853256, -0.02403501607477665, 0.003213947406038642, -0.0012942177709192038, -0.027157776057720184, -0.0020861062221229076, 0.02233285643160343, -0.027848878875374794, 0.0245981365442276, 0.013060561381280422, 0.0014205999905243516, 0.007230981718748808, -0.006610269192606211, 0.004613750614225864, 0.007909286767244339, -0.018429405987262726, -0.02083546668291092, 0.00972663052380085, 0.048837922513484955, 0.016048939898610115, 0.019376471638679504, 0.012497439980506897, -0.022857582196593285, -0.04571516439318657, -0.014103613793849945, 0.017495136708021164, -0.020464317873120308, -0.04543360322713852, -0.005298453848809004, 0.006917425896972418, -0.03419678658246994, 0.012062301859259605, -0.009745827876031399, 0.0046233488246798515, -0.00955385435372591, -0.01867257058620453, 0.019811609759926796, -0.004069827031344175, 0.011409593746066093, -0.004719335585832596, 0.022831985726952553, -0.019338076934218407, 0.0038170625921338797, -0.01988839916884899, 0.022921573370695114, 0.0254428181797266, -0.00045713564031757414, -0.05086003988981247, 0.02766970358788967, 0.030613290145993233, -0.02638988383114338, -0.028693560510873795, 0.020541107282042503, -0.0025436419527977705, 0.031867511570453644, -0.031739529222249985, -0.009547456167638302, 0.03900890797376633, -0.03179072216153145, 0.021859321743249893, 0.008529999293386936, 0.00134941004216671, 0.01192152127623558, -0.05293334648013115, -0.012849390506744385, -0.007461349479854107, -0.001938926987349987, 0.00867717806249857, -0.013898842968046665, 0.026364287361502647, -0.013758063316345215, 0.008114057593047619, -0.006130336783826351, 0.0003615490859374404, 0.03831780329346657, 0.01493549719452858, 0.019146103411912918, -0.0029211887158453465, 0.026441076770424843, -0.0062423208728432655, 0.0047705285251140594, 0.0037754683289676905, 0.04840278625488281, 0.00468414044007659, -0.019184498116374016, -0.02820722758769989, 0.021948909386992455, 0.02293437160551548, 0.029973380267620087, 0.01897972822189331, 0.013169345445930958, -0.00044593721395358443, 0.024700522422790527, -0.0025084468070417643, -0.01822463423013687, -0.008338025771081448, -0.00012068300566170365, 0.05339408293366432, -0.01672724448144436, -0.018429405987262726, 0.01571618765592575, -0.004287396557629108, -0.013041364029049873, -0.001624571275897324, -0.020656291395425797, -0.036782022565603256, -0.025558002293109894, -0.03463192284107208, -0.011492782272398472, -0.061073001474142075, -0.015792977064847946, 0.029512643814086914, 0.01403962355107069, 0.04530562087893486, -0.0319954939186573, -0.017047200351953506, -0.010500921867787838, 0.004057028796523809, -0.05262618884444237, 0.04069827124476433, 0.05155114084482193, 0.006783044897019863, -0.034683115780353546, -0.04154295101761818, -0.04768608510494232, 0.011812737211585045, 0.01938926987349987, 0.021104227751493454, 0.01995239034295082, 0.02600593864917755, -0.03330091014504433, -0.025596395134925842, 0.026031535118818283, 0.00872197188436985, 0.009438671171665192, 0.014385174959897995, -0.016778437420725822, 0.04269478842616081, 0.03693559765815735, 0.0027836081571877003, -0.036910004913806915, 0.017405549064278603, 0.04297634959220886, 0.0039610425010323524, 0.03793385997414589, 0.022947169840335846, -0.03959762305021286, -0.002810804173350334, -0.004799324087798595, 0.00736536318436265, -0.009592249058187008, 0.0007822898332960904, -0.009246697649359703, -0.00694302236661315, -0.008248439058661461, -0.0028699960093945265, 0.013399713672697544, -0.01694481447339058, 0.02933347038924694, -0.0012838192051276565, -0.00903552770614624, 0.013566089794039726, 0.014423568733036518, 0.036910004913806915, -0.045510392636060715, -0.006095141638070345, 0.019184498116374016, 0.0060695456340909, 0.014845909550786018, 0.03278898447751999, 0.008043667301535606, 0.007416555657982826, 0.038420189172029495, -0.030766867101192474, 0.01766151376068592, -0.03414559364318848, 0.006120738107711077, -0.0383433997631073, -0.025711579248309135, -0.019683629274368286, 0.002479650778695941, -0.008152452297508717, 0.0020605099853128195, 0.014257192611694336, -0.0035930941812694073, 0.015370636247098446, 0.0036666837986558676, 0.03780587762594223, -0.013502098619937897, 0.005352846346795559, 0.00652708113193512, 0.008248439058661461, 0.007685318123549223, 0.010724890045821667, -0.01938926987349987, -0.00662946654483676, -0.0075189415365457535, 0.010206562466919422, 0.005237662699073553, -0.043155524879693985, -0.021321797743439674, -0.013668475672602654, -0.028463192284107208, 0.013655677437782288, -0.003324331948533654, -0.02148817479610443, -0.019504453986883163, 0.010910463519394398, -0.007083802483975887, 0.00809486024081707, 0.017866283655166626, 0.009336285293102264, 0.015165864489972591, -0.006296713370829821, -0.020374732092022896, 0.005762388464063406, -0.018339818343520164, 0.005851976107805967, -0.001732556032948196, 0.013502098619937897, 0.0286167711019516, -0.011435190215706825, 0.007032610010355711, -0.016650455072522163, -0.02098904550075531, 0.002441256307065487, -0.0017037601210176945, -0.0014142008731141686, 0.000483131967484951, 0.0031243599951267242, 0.037037983536720276, -0.02585235983133316, 0.017431145533919334, -0.002610832452774048, -0.03708917647600174, 0.010827275924384594, -0.011761544272303581, 0.014410770498216152, 0.014103613793849945, 0.004121019970625639, -0.0002337670885026455, -0.04046790301799774, -0.011851130984723568, -0.020656291395425797, -0.018365414813160896, -0.01707279682159424, 0.01160796545445919, 0.014218797907233238, 0.016202518716454506, 0.01813504658639431, 0.007947680540382862, 0.015101873315870762, -0.00026316294679418206, 0.04602231830358505, 0.025020476430654526, 0.0077109141275286674, -0.005164072848856449, -0.010513720102608204, 0.0010686495807021856, -0.019875600934028625, -0.006277516018599272, 0.025186853483319283, 0.002540442394092679, -0.029256680980324745, 0.04958022013306618, 0.019005324691534042, -0.015741784125566483, -0.010929660871624947, 0.004936905112117529, 0.00025816363631747663, -0.017802294343709946, -0.013182143680751324, -0.03754991292953491, 0.032328248023986816, 0.003903450444340706, -0.016074536368250847, -0.018301423639059067, 0.014397973194718361, 0.023023957386612892, 0.017303163185715675, 0.015101873315870762, 0.0036186904180794954, -0.0026300298050045967, -0.006498285103589296, -0.003356327535584569, 0.006536679808050394, 0.0686495378613472, 0.007096600718796253, -0.04033992066979408, -0.016842428594827652, 0.018032660707831383, -0.009221102111041546, 0.01891573704779148, 0.03417119011282921, -0.014257192611694336, -0.000356149859726429, -0.03378724306821823, 0.016637656837701797, 0.04141496866941452, 0.022448038682341576, 0.013950035907328129, -0.030408518388867378, 0.012970973737537861, 0.0118767274543643, 0.01709839329123497, -0.0006351105985231698, 0.02563478983938694, -0.028284016996622086, 0.040928635746240616, -0.011140831746160984, -0.013297327794134617, -0.008997133001685143, 0.019824407994747162, -0.007998873479664326, 0.0054104384034872055, 0.0007051007123664021, 0.010116975754499435, 0.009982594288885593, 0.015396231785416603, -0.013092556968331337, -0.010827275924384594, 0.022102488204836845, 0.021846523508429527, -0.03598853200674057, -0.018749359995126724, 0.007115798071026802, 0.001280619646422565, -0.016842428594827652, -0.044384151697158813, -0.01778949610888958, 0.01135840080678463, 0.04474249854683876, 0.026722637936472893, -0.00271001853980124, 0.031176410615444183, 0.04671342298388481, -0.028539981693029404, 0.015204259194433689, 0.004021833650767803, 0.014884304255247116, -0.02014436386525631, 0.0033051345963031054, 0.007026210892945528, -0.026492269709706306, -0.006210325751453638, 0.004821720998734236, -0.0028507986571639776, 0.03458072990179062, -0.07069724798202515, 0.00303477281704545, -0.006802242249250412, -0.02864236757159233, -0.002655626041814685, 0.023996621370315552, -0.017546329647302628, 0.025993140414357185, 0.002774009481072426, -0.014359578490257263, -0.0021069033537060022, -0.006453491281718016, -0.030690079554915428, 0.010449728928506374, 0.008958738297224045, -0.03644926846027374, 0.0037178765051066875, -0.017469540238380432, -0.019402068108320236, 0.04376983642578125, -0.01286218874156475, -0.020592300221323967, -0.023075150325894356, -0.032456230372190475, -0.009054725058376789, -0.010923261754214764, -0.008549196645617485, 0.014269990846514702, 0.026018736883997917, 0.013361318968236446, 0.032609809190034866, 0.039827991276979446, 0.013374117203056812, -0.01584417000412941, 0.030408518388867378, -0.03335210308432579, -0.017136787995696068, 0.010769683867692947, -0.016202518716454506, -0.02695300430059433, 0.036193303763866425, 0.014461963437497616, 0.00924029853194952, 0.017149586230516434, -0.014321183785796165, 0.0009862611768767238, -0.012273471802473068, 0.018506193533539772, -0.015664994716644287, -0.012427050620317459, -0.006783044897019863, 0.012727808207273483, -0.017738303169608116, -0.03401761129498482, -0.044281765818595886, -0.0503225140273571, -0.0034907085355371237, -0.03350568190217018, -0.00767891900613904, -0.04965700954198837, 0.00845320988446474, -0.012747005559504032, -0.0005667202058248222, 0.040800657123327255, 0.017469540238380432, 0.04213166609406471, 0.01867257058620453, -0.026620252057909966, 0.010411334224045277, 0.01763591729104519, -0.011070441454648972, 0.0012270272709429264, -0.04318112134933472, -0.01535783801227808, 0.029359066858887672, -0.03611651435494423, 0.027976861223578453, 0.01135840080678463, 0.004034631885588169, 0.01123041845858097, -0.006421495694667101, -0.01897972822189331, -0.0062199244275689125, -0.010436930693686008, 0.0051800706423819065, -0.02356148324906826, -0.012951776385307312, -0.010520119220018387, 0.03629568964242935, 0.020694686099886894, 0.0011262414045631886, -0.022512029856443405, -0.01750793494284153, -0.0034299171529710293, -0.022064093500375748, 0.009675437584519386, -0.005103281233459711, 0.005256860051304102, 0.013374117203056812, 0.018749359995126724, -0.03895771503448486, -0.004338589031249285, 0.0017309562535956502, 0.002729215659201145, -0.018275827169418335, -0.028975119814276695, -0.008869151584804058, -0.02211528643965721, -0.008664379827678204, -0.023958226665854454, 0.018109450116753578, 0.008402016945183277, -0.027464933693408966, 0.0019469258841127157, -0.007275775540620089, -0.002684422070160508, 0.006559076253324747, -0.011211221106350422, -0.03281458094716072, -0.009419473819434643, -0.0034491142723709345, 0.015856968238949776, -0.011614364571869373, 0.043155524879693985, -0.02444455772638321, -0.03849697858095169, 0.010097778402268887, -0.016650455072522163, 0.013092556968331337, -0.010321746580302715, -0.008011671714484692, -0.010609705932438374, -0.003663484239950776, 0.011857530102133751, -0.02820722758769989, -0.05979318171739578, -0.012087898328900337, -0.008293231949210167, -0.003324331948533654, -0.03836899623274803, -0.03573256731033325, 0.0042745983228087425, -0.02751612663269043, -0.013591686263680458, 0.005800783168524504, 0.010449728928506374, -0.0008942740969359875, -0.015984948724508286, -0.008709173649549484, -0.009982594288885593, 0.014410770498216152, -0.024930890649557114, 0.010059383697807789, 0.029794204980134964, 0.008600388653576374, 0.0360141284763813, -0.0009334685746580362, 0.013604484498500824, 0.010968055576086044, -0.010257755406200886, 0.011326405219733715, 0.009541057050228119, 0.0030907648615539074, 0.01192152127623558, 0.022012900561094284, -0.0004863315261900425, 0.018659772351384163, -0.02233285643160343, 0.03580935671925545, 0.03429917246103287, -0.0031339586712419987, -0.002114902250468731, 0.02183372527360916, 0.007269376423209906, -0.04643186181783676, -0.0032155471853911877, -0.006603870075196028, -0.006402298342436552, -0.012683014385402203, -0.010539315640926361, 0.0024300578515976667, 0.012542233802378178, -0.03417119011282921, -0.0017869484145194292, 0.0032347445376217365, -0.0004223405267111957, 0.010225759819149971, 0.030818060040473938, -0.009272294119000435, 0.0034811098594218493, 0.015153066255152225, -0.012132692150771618, 0.02606992982327938, 0.0354510098695755, -0.02412460371851921, 0.0069558206014335155, -0.025596395134925842, 0.011006450280547142, -0.040519095957279205, 0.03519504517316818, 0.06240401417016983, -0.013924439437687397, 0.03711477294564247, 0.013169345445930958, -0.026722637936472893, -0.006591071840375662, -0.01493549719452858, 0.02531483583152294, -0.05027132108807564, 0.011140831746160984, -0.0004267399199306965, -0.02083546668291092, -0.032174669206142426, 0.008984334766864777, -0.007749308831989765, -0.0015125870704650879, 0.010456128045916557, 0.019747620448470116, -0.016074536368250847, 0.024239787831902504, -0.0434882752597332, -0.011851130984723568, 0.046892598271369934, -0.01951725222170353, -0.007653322536498308, 0.01562660001218319, -0.005864774342626333, 0.0025484412908554077, 0.017776697874069214, 0.0066422647796571255, -0.007083802483975887, -0.01703440211713314, 0.015613801777362823, 0.058718133717775345, 0.026364287361502647, 0.02820722758769989, 0.014257192611694336, 0.000834282545838505, 0.0030027772299945354, 0.030690079554915428, -0.006597470957785845, 0.02818163111805916, 0.018621377646923065, 0.04627828299999237, 0.008043667301535606, 0.0006751049659214914, -0.04673901945352554, -0.017367154359817505, 0.016522472724318504, 0.01688082329928875, -0.02130899950861931, 0.006491885986179113, 0.022729599848389626, -0.025058871135115623, 0.0021117026917636395, 0.020067574456334114, 0.010091379284858704, -0.0030011774506419897, -0.0004715335962828249, -0.02353588677942753, -0.005077685229480267, -0.01569059118628502, 0.021027440205216408, 0.00983541551977396, 0.005490426905453205, -0.05866694077849388, 0.0025564399547874928, 0.021756935864686966, 0.01055851299315691, 0.0030827659647911787, 0.010596907697618008, 0.013668475672602654, 0.001511787180788815, -0.008670778945088387, 0.022921573370695114, 0.01709839329123497, 0.01118562463670969, 0.01681683212518692, 0.003865055739879608, 0.0034235180355608463, 0.030178150162100792, 0.013310126028954983, -0.023267123848199844, -0.002866796450689435, 0.02042592316865921, 0.011492782272398472, -0.03337769955396652, 0.03990478068590164, -0.008741169236600399, 0.014282789081335068, -0.05395720154047012, -0.017904678359627724, 0.0033467288594692945, -0.032584212720394135, 0.04394901171326637, -0.0028683962300419807, -0.05073205754160881, 0.006287114694714546, -0.04420497640967369, -0.00033375300699844956, -0.018275827169418335, 0.026313094422221184, -0.002434857189655304, 0.02535323053598404, -0.0063223098404705524, -0.04860755428671837, 0.026594655588269234, 0.036500461399555206, -0.022614415735006332, -0.0010014589643105865, 0.016650455072522163, -0.00370187871158123, -0.0009006731561385095, -0.008523600175976753, 0.0121390912681818, 0.001015856978483498, -0.005874373018741608, -0.016023343428969383, 0.01700880564749241, -0.014218797907233238, -0.008350824005901814, -0.010027388110756874, 0.0063095116056501865, 0.01599774695932865, 0.005205667112022638, 0.014679533429443836, 0.00999539252370596]}>, <Node element_id='4:c66ce224-058b-46a0-b519-9d2e36be1920:4244' labels=frozenset({'__Entity__', 'Concept'}) properties={'human_readable_id': 3065, 'name': 'EVENTS OF LIFE', 'description': "The events of life encompass the joys, sorrows, virtues, sins, hatreds, jealousies, and loves that shape the characters' experiences and histories.", 'id': '8d81de91867e4086bf01670e2fc9becd', 'description_embedding': [0.03977350890636444, 0.01900043524801731, 0.01564350724220276, 0.02924848534166813, -0.02098357118666172, 0.0034677181392908096, 0.005738907027989626, -0.018180591985583305, 0.02455100044608116, 0.032594334334135056, 0.01926633156836033, -0.010397614911198616, -0.042055774480104446, -0.03263865038752556, 0.027232112362980843, 0.019775964319705963, -0.009599928744137287, 0.0782175362110138, 0.007727582473307848, 0.021338099613785744, 0.06266266107559204, 0.012153632007539272, 0.002927618334069848, 0.038488347083330154, -0.027719587087631226, -0.027896849438548088, -0.00860282126814127, 0.06146613508462906, 0.03959624469280243, 0.005971565842628479, 0.025215739384293556, -0.015211427584290504, 0.04852589592337608, -0.009572231210768223, 0.039972927421331406, 0.037757135927677155, 0.04954516142606735, -0.026412267237901688, 0.024883370846509933, -0.0017878690268844366, -0.0057333679869771, -0.021027887240052223, -0.06598635762929916, 0.022534627467393875, -0.00916230957955122, -0.0018003329169005156, -0.011378103867173195, 0.015111717395484447, -0.03935250639915466, 0.07569153606891632, -0.033170439302921295, -0.03232843801379204, 0.040416087955236435, 0.029758118093013763, -0.009887982159852982, -0.0013384782942011952, -0.021681547164916992, 0.012785132974386215, -0.04015019163489342, -0.006857883185148239, -0.005132333375513554, -0.014314031228423119, -0.008287070319056511, 0.035851553082466125, -0.04349604249000549, -0.013771161437034607, -0.014070293866097927, -0.002247646450996399, -0.014735031872987747, 0.010585957206785679, 0.03662708029150963, 0.02867237851023674, -0.015167112462222576, 0.014214320108294487, -0.012563553638756275, 0.01767095923423767, -0.017814986407756805, 0.04433804377913475, -0.018956121057271957, -0.03771281987428665, 0.06430234760046005, 0.02200283668935299, -0.01782606542110443, 0.033148281276226044, -0.010525022633373737, 0.002888841787353158, -0.020894940942525864, 0.01188773661851883, -0.025814004242420197, 0.023598209023475647, -0.010874010622501373, -0.018391093239188194, -0.013494187034666538, 0.03815597668290138, 0.032417070120573044, -0.012541395612061024, 0.025038475170731544, -0.03961840271949768, -0.027409374713897705, 0.037003763020038605, 0.01485690101981163, -0.03474365547299385, -0.0008136119577102363, 0.024174315854907036, 0.00968302134424448, 0.03443344309926033, -0.028428640216588974, -0.03197391331195831, -0.059338971972465515, -0.017837144434452057, -0.12142552435398102, -0.02319936640560627, -0.019011514261364937, 0.06235245242714882, -0.009804889559745789, 0.0008205363410525024, 0.019510067999362946, 0.03088817186653614, 0.03990645706653595, -0.011776946485042572, -0.012087157927453518, 0.0532233789563179, -0.013405555859208107, -0.02075091376900673, 0.01749369688332081, 0.008652676828205585, 0.016119902953505516, -0.0017130859196186066, -0.004146304912865162, -0.02455100044608116, 0.008713611401617527, -0.004290331620723009, 0.016928669065237045, -0.08065491169691086, -0.008348004892468452, -0.03161938488483429, -0.015045243315398693, 0.04821568354964256, -0.01913338340818882, 0.019543305039405823, -0.016540903598070145, -0.06541024893522263, 0.009406046941876411, -0.010037547908723354, 0.026212846860289574, -0.015233585610985756, 0.003487106179818511, 0.02570321410894394, 0.0036809882149100304, -0.03458854928612709, 0.020739834755659103, 0.03961840271949768, -0.009156770072877407, 0.010718905366957188, 0.05366653949022293, -0.06089002639055252, 0.004858129192143679, -0.024484526365995407, -0.0561925433576107, 0.018025485798716545, 0.03578507900238037, -0.002498307963833213, -0.004589464049786329, 0.01240844838321209, -0.026500899344682693, -0.018313540145754814, -0.050918951630592346, 0.029049063101410866, -0.015133875422179699, -0.05632549151778221, -0.0229334719479084, -0.013250449672341347, -0.05747770518064499, 0.0547301210463047, -0.001213839859701693, -0.012663264758884907, 0.02410784177482128, 0.019842438399791718, -0.03977350890636444, 0.023110734298825264, -0.0024179855827242136, 0.0006678542704321444, -0.0458226278424263, 0.0009901830926537514, 0.01785930246114731, 0.035386234521865845, 0.011062352918088436, 0.051938217133283615, -0.0517609566450119, -0.056945912539958954, 0.031131910160183907, 0.001951283891685307, -0.0317523330450058, -0.008253834210336208, 0.04963379353284836, 0.02233520708978176, -0.0042487857863307, 0.025393003597855568, -0.011865578591823578, 0.018723461776971817, 0.01515603344887495, -0.04068198427557945, 0.05122916400432587, -0.04052687808871269, -0.022301970049738884, 0.021515361964702606, 0.022301970049738884, 0.016773562878370285, 0.015466243959963322, -0.028273535892367363, 0.009289717301726341, -0.024440210312604904, 0.012607869692146778, -0.035231128334999084, 0.0273429024964571, 0.04254325106739998, -0.007688806392252445, 0.024927685037255287, -0.030799541622400284, 0.004063212778419256, 0.016252851113677025, 0.016651693731546402, 0.06877825409173965, 0.020961415022611618, -0.01751585304737091, 0.04090356454253197, -0.007578016724437475, -0.032727282494306564, -0.011998525820672512, -0.057654969394207, -0.0206955187022686, 0.013505266048014164, -0.047816842794418335, -0.01866806671023369, -0.015277901664376259, -0.06922141462564468, -0.021880969405174255, 0.016662772744894028, -0.04105866700410843, 0.033303387463092804, -0.037469081580638885, 0.05375517159700394, -0.023155050352215767, 0.04092572256922722, 0.0035092642065137625, 0.029647327959537506, 0.02570321410894394, 0.0218144953250885, 0.01197636779397726, -0.009278638288378716, 0.03146427869796753, -0.0054619330912828445, -0.009854745119810104, -0.029226327314972878, 0.026811111718416214, 0.0036865277215838432, 0.03904229402542114, -0.028761010617017746, 0.027963323518633842, 0.021260546520352364, 0.07843911647796631, 0.015610271133482456, -0.042786989361047745, 0.042476776987314224, -0.015012006275355816, -0.02674463763833046, -0.004306950140744448, 0.0021687087137252092, -0.001611990388482809, -0.024307264015078545, 0.009921219199895859, -0.047063469886779785, -0.03742476552724838, -0.026367953047156334, -0.04015019163489342, -0.014867980033159256, -0.018524039536714554, 0.04830431565642357, 0.05096326768398285, 0.0812753364443779, 0.025747530162334442, -0.009079216979444027, 0.037158869206905365, -0.03157506883144379, -0.042631883174180984, 0.018911805003881454, 0.013084265403449535, -0.03223980590701103, 0.013571740128099918, 0.016142060980200768, 0.004420509561896324, -0.00823721569031477, -0.015998035669326782, -0.004489753395318985, 0.051938217133283615, 0.00871915090829134, 0.03594018518924713, 0.004957839846611023, 0.059117391705513, 0.02277836576104164, 0.004223858006298542, -0.05486306548118591, 0.007118239067494869, 0.023420946672558784, -0.016197456046938896, 0.07950270175933838, 0.01351634506136179, -0.04812705144286156, 0.04287562146782875, 0.009627626277506351, 0.045157887041568756, 0.02836216799914837, 0.06279560923576355, -0.019044751301407814, -0.005392689257860184, -0.0018972738180309534, -0.004655937664210796, 0.07072815299034119, 0.05198253318667412, 0.00037356908433139324, -0.005057550501078367, -0.017870381474494934, -0.02260110154747963, 0.008841019123792648, 0.0037446923088282347, -0.0050852480344474316, -0.022135784849524498, 0.035541340708732605, -0.023886263370513916, 0.001463809167034924, -0.007683266885578632, -0.059206023812294006, -0.05778791382908821, 0.026190688833594322, 0.025459475815296173, -0.02512710727751255, 0.02408568374812603, -0.02971380203962326, 0.007849451154470444, -0.024728264659643173, -0.0024096763227134943, 0.0030882633291184902, -0.04032745584845543, -0.011566446162760258, -0.01678464189171791, 0.007356436923146248, 0.0009112454135902226, -0.023576050996780396, -0.054021064192056656, 0.014967690221965313, -0.02794116549193859, -0.0007602944388054311, -0.05552780628204346, -0.0037086857482790947, -0.032151173800230026, -0.008569584228098392, -0.018512960523366928, 0.0008897798834368587, -0.0012581556802615523, 0.03368007391691208, -0.008591742254793644, -0.05787654593586922, 0.03975135087966919, 0.046620313078165054, -0.027298586443066597, -0.017715275287628174, -0.02335447259247303, 0.010131719522178173, -0.0038111661560833454, -0.009095835499465466, 0.037291817367076874, -0.05397674813866615, -0.0017920236568897963, 0.0057278284803032875, 0.003149197669699788, 0.05220411345362663, -0.04154614359140396, -0.043385252356529236, -0.0517166405916214, -0.017105931416153908, -0.07041794061660767, 0.023110734298825264, 0.027586638927459717, -0.015200348570942879, 0.0317966490983963, 0.04050472006201744, 0.00026018271455541253, -0.008791163563728333, -0.011095589958131313, -0.009987693279981613, 0.062130872160196304, -0.01759340614080429, -0.014380505308508873, -0.019587621092796326, 0.01597587764263153, -0.03933034837245941, 0.029758118093013763, 0.0856626108288765, 0.02822921983897686, -0.03166370093822479, 0.005398228764533997, -0.00897950679063797, -0.005187728442251682, -0.004794424865394831, 0.0034123233053833246, -0.04050472006201744, 0.01631932519376278, 0.011511051096022129, -0.009411586448550224, 0.0022698042448610067, 0.010076324455440044, 0.0036034355871379375, 0.013228291645646095, -0.004046594258397818, 0.024861212819814682, -0.026079898700118065, 0.008071030490100384, -0.024129999801516533, 0.036582764238119125, 0.036405500024557114, 4.221693961881101e-05, -0.006348250899463892, -0.004473134875297546, 0.007062844466418028, 0.020795229822397232, 0.0065088956616818905, 0.02348741888999939, 0.01579861342906952, -0.05552780628204346, 0.004165693186223507, -0.031109752133488655, 0.03766850382089615, -0.02984674833714962, -0.006298395339399576, 0.0339902862906456, -0.035519182682037354, 0.027099164202809334, -0.015654586255550385, -0.023420946672558784, -0.03328122943639755, -0.006226382218301296, 0.0013163202675059438, 0.04715210199356079, 0.02780821919441223, -0.037912242114543915, -0.0065088956616818905, -0.045733995735645294, 0.0015787534648552537, 0.01299563329666853, 0.01158860418945551, 0.04595557227730751, 0.023841947317123413, -0.007201331667602062, -0.010896168649196625, -0.06159908324480057, -0.004639319144189358, -0.025082791224122047, -0.008281530812382698, 0.01757124811410904, -0.03443344309926033, -0.0709054172039032, 0.012009604834020138, 0.018956121057271957, -0.00473072100430727, 0.010602575726807117, 0.039839982986450195, 0.006082355510443449, -0.048259999603033066, -0.0019305107416585088, -0.011632920242846012, -0.008619439788162708, 0.008475413545966148, 0.009095835499465466, -0.0044038910418748856, 0.03789008408784866, 0.019454674795269966, -0.0037502318155020475, -0.023265840485692024, -0.013405555859208107, 0.009284177795052528, -0.004423279315233231, 0.06120023876428604, 0.013183976523578167, 0.003697606734931469, -0.004010587465018034, -0.029159853234887123, -0.020130490884184837, -0.01603127270936966, 0.0008129195193760097, 0.011222997680306435, 0.009732876904308796, -0.03904229402542114, -0.004085370805114508, -0.008591742254793644, 0.051051899790763855, -0.030068328604102135, -0.0022933471482247114, 0.004093680065125227, 0.00952791515737772, -0.03102112002670765, -0.0018668066477403045, 0.03266080841422081, -0.014081372879445553, 0.05885149538516998, 0.014469136483967304, 0.028583746403455734, 0.03456639125943184, -0.0043761939741671085, 0.015831850469112396, 0.057433389127254486, -0.03412323072552681, 0.02189204841852188, 0.010403154417872429, 0.011776946485042572, 0.002023297129198909, -0.04221088066697121, -0.04147966951131821, -0.007683266885578632, -0.021980678662657738, -0.008957348763942719, -0.004858129192143679, -0.012873765081167221, 0.020496096462011337, 0.001108589582145214, 0.018047643825411797, -0.04843726381659508, 0.012696501798927784, -0.02512710727751255, 0.00834246538579464, -0.045091412961483, -0.004968918859958649, -0.002566166687756777, 0.018313540145754814, 0.006918817758560181, -0.022822681814432144, 0.055483490228652954, 0.027852535247802734, -0.008475413545966148, 0.01608666591346264, 0.01949898898601532, -0.002883302280679345, 0.03751339763402939, 0.03368007391691208, 0.053444959223270416, -0.004511910956352949, 0.026523057371377945, -0.01803656481206417, -0.006442422047257423, -0.021382415667176247, 0.012076078914105892, 0.02554810792207718, -0.010137259028851986, 0.021127598360180855, -0.024794738739728928, -0.010896168649196625, 0.034522075206041336, 0.002236567437648773, -0.003470487892627716, -0.036405500024557114, 0.029049063101410866, -0.04015019163489342, -0.03239491209387779, 0.04420509561896324, 0.011954210698604584, -0.009948916733264923, -0.016153139993548393, 0.034499917179346085, 0.03458854928612709, -0.015167112462222576, 0.010890629142522812, -0.00012887960474472493, 0.009965535253286362, 0.02439589612185955, 0.001142518944106996, -0.021437810733914375, -0.01090724766254425, -0.003154737176373601, -0.0065476722083985806, -0.0651000365614891, -0.03188528120517731, -0.011799104511737823, -0.050786007195711136, -0.044249411672353745, -0.017117010429501534, -0.0767994299530983, 0.025658898055553436, 0.028140587732195854, 0.01892288401722908, 0.003381856018677354, -0.024595316499471664, -0.006364868953824043, 0.0036311328876763582, 0.0002079038240481168, 0.021670468151569366, -0.0055533344857394695, -0.007229028735309839, 0.043052881956100464, 0.021227309480309486, -0.0199643075466156, -0.020551491528749466, -0.004658707417547703, 0.047063469886779785, -0.006747093517333269, -0.013472029007971287, 0.0033181519247591496, -0.017615564167499542, -0.03000185452401638, -0.013937345705926418, -0.006769251544028521, -0.020296676084399223, 0.017504775896668434, -0.02171478420495987, -0.0033236914314329624, 0.009151230566203594, -0.002023297129198909, -0.011488893069326878, 0.013472029007971287, -0.025503791868686676, 0.012951317243278027, -0.023576050996780396, -0.010984799824655056, 0.01822490803897381, 0.024307264015078545, 0.02304426021873951, 0.02421863190829754, 0.003196283243596554, -0.034943077713251114, 0.01038653589785099, 0.012076078914105892, 0.005043701734393835, 0.002080076839774847, -0.0022864227648824453, 0.0015953718684613705, -0.0014901217073202133, 0.0060214209370315075, 0.0066196853294968605, -0.009001664817333221, 0.0014402662636712193, -0.03520897030830383, -0.025636740028858185, 0.0347214974462986, 0.019399279728531837, 0.031708016991615295, -0.013095344416797161, 0.009771652519702911, 0.0011099744588136673, -0.00584415765479207, 0.040726300328969955, -0.004564536269754171, 0.0009486369090154767, 0.023908421397209167, -0.02776390314102173, 0.010098482482135296, -0.011378103867173195, -0.008558505214750767, -0.011854499578475952, -0.029004747048020363, -0.031840965151786804, 0.030046170577406883, 0.006154368631541729, 0.034344810992479324, -0.0017490925965830684, 0.04210009053349495, 0.0027392758056521416, 0.027409374713897705, 0.032572176307439804, -0.004595003556460142, -0.017338590696454048, -0.01203176286071539, 0.047683894634246826, 0.01965409517288208, 0.05588233098387718, 0.03844403102993965, 0.022146863862872124, 0.025016317144036293, 0.011710473336279392, -0.017272116616368294, -0.011721551418304443, -0.012707579880952835, 0.01329476572573185, 0.016219614073634148, -0.014613163657486439, -0.01894504204392433, -0.022401681169867516, 0.01978704333305359, -0.00916230957955122, 0.045733995735645294, -0.00920108612626791, 0.008558505214750767, 0.014668558724224567, -0.03607312962412834, 0.014613163657486439, -0.03015696071088314, 0.0065088956616818905, 0.02306641824543476, -0.040283139795064926, -0.009583310224115849, 0.0002388903230894357, -0.010342219844460487, -0.022146863862872124, 0.02494984306395054, 0.002838986460119486, -0.02986890636384487, -0.02051825448870659, -0.04526867717504501, -0.010380996391177177, 0.03512033820152283, 0.030622277408838272, 0.010469628497958183, -0.054154012352228165, 0.023553892970085144, 0.014613163657486439, -0.008375702425837517, 0.04436020180583, -0.03899797797203064, 0.0236203670501709, -0.03128701448440552, 0.010209272615611553, -0.02189204841852188, 0.011355945840477943, -0.002618792001157999, 0.02924848534166813, -0.027165638282895088, -0.04460394009947777, -0.019089067354798317, -0.004877516999840736, 0.02971380203962326, -0.007478305604308844, -0.04830431565642357, -0.0015732139581814408, -0.009295256808400154, 0.021670468151569366, 0.005749986041337252, 0.007511542644351721, 0.0020689978264272213, 0.006952054798603058, -0.010785378515720367, 0.0003939959278795868, -0.009838126599788666, 0.021005729213356972, -0.011001418344676495, 0.002674186835065484, -0.0035397314932197332, 0.024041367694735527, 0.010597036220133305, 0.0018488033674657345, -0.030201276764273643, -0.00275312433950603, -0.008087649010121822, 0.007943622767925262, 0.010353298857808113, 0.00587739422917366, -0.015865087509155273, -0.04644304886460304, 0.039839982986450195, -0.015012006275355816, -0.0004116530471947044, 0.01337231881916523, 0.012884844094514847, -0.01905583031475544, 0.01707269437611103, 0.022977786138653755, 0.01936604268848896, -0.019476832821965218, -0.018136275932192802, 0.018546197563409805, 0.04303072392940521, -0.012009604834020138, 0.02137133665382862, -0.004196160472929478, -0.03503170609474182, 0.001629993668757379, -0.006043578963726759, -0.002557857660576701, 0.015565955080091953, -0.008281530812382698, -0.014446978457272053, 0.004354035947471857, -0.0013917958131060004, -0.026700321584939957, 0.0032682965975254774, -0.03239491209387779, 0.017992250621318817, -0.0016687700990587473, -0.00598818389698863, -0.010840773582458496, -0.008364623412489891, -0.0047611878253519535, -0.024329422041773796, -0.04294209182262421, 0.012308737263083458, -0.014890138059854507, -0.002764203352853656, -0.014136767946183681, 0.007428450509905815, -0.017604485154151917, 0.013350160792469978, -0.0546414889395237, 0.020473938435316086, -0.006004802417010069, -0.017781749367713928, -0.004387272521853447, -0.01918877847492695, 0.030179118737578392, -0.0032682965975254774, -0.0019582081586122513, -0.03456639125943184, 0.04365114867687225, -0.015144954435527325, 0.0092398626729846, -0.02512710727751255, -0.001148058450780809, -0.0203299131244421, 0.016895432025194168, 0.007422911003232002, -0.04429372772574425, -0.008852098137140274, 0.03463286533951759, -0.0025910944677889347, -0.013195055536925793, 0.020784150809049606, -0.0332147553563118, 0.018878567963838577, -0.0038194754160940647, -0.01046408899128437, 0.03862129524350166, -0.0055976505391299725, -0.0050658597610890865, -0.030400697141885757, -0.007589095272123814, -0.015842929482460022, 0.023088576272130013, -0.022324128076434135, -0.0018709612777456641, 0.013638214208185673, -0.02971380203962326, -0.030245592817664146, 0.0014901217073202133, 0.03828892484307289, 0.029447905719280243, -0.0013945655664429069, 0.013416634872555733, -0.022634338587522507, -0.03800087049603462, 0.04183419793844223, 0.0013689454644918442, 0.004002278670668602, 0.005376070737838745, 0.0046503981575369835, 0.07436205446720123, 0.009406046941876411, 0.01686219498515129, 0.037025921046733856, -0.02439589612185955, -0.017549090087413788, -0.007555858697742224, -0.02381978929042816, 0.010929405689239502, -0.023886263370513916, -0.01928848959505558, 0.01967625319957733, -0.0203299131244421, -0.01988675445318222, -0.0005497939419001341, 0.021249467507004738, -0.013361239805817604, 0.016308246180415154, 0.0019997544586658478, -0.021072203293442726, -0.021471045911312103, -0.006985291372984648, -0.0003169624542351812, 0.026545215398073196, -0.007500463631004095, 0.00532344589009881, -0.006387026980519295, -0.01913338340818882, 0.00793808326125145, 0.0050658597610890865, 0.026944058015942574, 0.033945970237255096, 0.02557026594877243, 0.015732139348983765, -0.011943131685256958, -0.03733613342046738, 0.00015796189836692065, 0.02246815338730812, -0.010641352273523808, -0.041169457137584686, 0.010369917377829552, -0.020606886595487595, -0.0024110611993819475, -0.02512710727751255, -0.03649413213133812, -0.0010365762282162905, -0.0062153032049536705, -0.005475781857967377, -0.018778856843709946, -0.0019706720486283302, 0.026545215398073196, 0.005478551611304283, -0.005619808100163937, -0.0046697864308953285, 0.0036754487082362175, -0.039219558238983154, -0.010912787169218063, 0.008663755841553211, 0.008580663241446018, -0.029204169288277626, 0.028605904430150986, 0.016308246180415154, -0.03372438997030258, -8.904290552891325e-06, 0.023310156539082527, 0.04283130541443825, 0.04608852043747902, -0.039684876799583435, -0.005575492512434721, 0.021748021245002747, -0.033325545489788055, 0.026966216042637825, 0.04418293759226799, -0.013926267623901367, 0.015842929482460022, -0.02231304906308651, -0.008442176505923271, 0.004522989969700575, 0.020418545231223106, 0.015554876066744328, -0.024285105988383293, -0.023753315210342407, -0.027896849438548088, -0.009217704646289349, -0.029647327959537506, 0.022091468796133995, 0.0487917922437191, 0.004414970055222511, 0.017061615362763405, -0.012231184169650078, -0.006796949077397585, -0.02393057756125927, -0.006348250899463892, 0.02971380203962326, -0.001895889057777822, 0.018933963030576706, -0.009882442653179169, -0.02512710727751255, 0.008791163563728333, 4.349361915956251e-05, -0.004237706772983074, -0.03516465425491333, 0.0026437195483595133, -0.009073677472770214, 0.026035582646727562, 0.04475904628634453, -0.01949898898601532, 0.005475781857967377, 0.005184958688914776, -0.0031215001363307238, -0.014380505308508873, -0.028871798887848854, 0.018656987696886063, 0.004129686858505011, -0.004398351535201073, -0.001587062724865973, -0.00034292880445718765, -0.020496096462011337, -0.014369426295161247, -0.020296676084399223, 0.01263002771884203, -0.019454674795269966, -0.04090356454253197, 0.02780821919441223, 0.016596298664808273, 0.03676002845168114, -0.009273098781704903, -0.012419527396559715, -0.010502864606678486, -0.019244173541665077, -0.039684876799583435, 0.021382415667176247, 0.025082791224122047, -0.0008863177499733865, 0.0011030500754714012, -0.023443102836608887, -0.037181027233600616, 0.01952114701271057, 0.007888227701187134, 0.006165447644889355, 0.030511487275362015, 0.0008454640046693385, -0.06957594305276871, -0.003229520283639431, -0.0009368655155412853, 0.022091468796133995, 0.0028639142401516438, 0.014502373524010181, 0.003908107057213783, 0.0002709154796320945, -0.010148338042199612, 0.0031381186563521624, -0.0177263543009758, 0.02614637278020382, 0.0007956086192280054, 0.01634148322045803, 0.037757135927677155, 0.012375211343169212, 0.003982890397310257, -0.018025485798716545, 0.011488893069326878, 0.04097003489732742, -0.00037806990440003574, 0.013106423430144787, -0.01678464189171791, 0.0025633969344198704, 0.009694100357592106, -0.010220351628959179, -0.002149320440366864, -0.049722425639629364, 0.030710909515619278, 0.02040746621787548, 0.0013668681494891644, 0.008696992881596088, 0.016075586900115013, 0.004716872237622738, -0.04008371755480766, 0.009433744475245476, 0.020296676084399223, -0.03239491209387779, -0.0042266277596354485, 0.009478060528635979, 0.030267750844359398, 0.010148338042199612, 0.018623750656843185, -0.00580538110807538, 0.02645658329129219, -0.04258756712079048, -0.01691759005188942, -0.031065436080098152, -0.019543305039405823, -0.024462368339300156, -0.0021133138798177242, -0.0027614335995167494, -0.03806734457612038, 0.006110052578151226, -0.007389673963189125, 0.02865022048354149, -0.0188453309237957, 0.0376906618475914, 0.05158369243144989, -0.00042931013740599155, 0.027719587087631226, 0.007361976429820061, -0.009489139541983604, 0.03707023710012436, 0.010657970793545246, -0.007832832634449005, -0.034344810992479324, 0.011328248307108879, 0.007561398204416037, -0.025681056082248688, -0.008336925879120827, -0.0033929350320249796, -0.012729737907648087, 0.014723952859640121, 0.0036532909143716097, 0.006270697806030512, -0.02030775509774685, 0.026545215398073196, 0.041457511484622955, -0.0032738361041992903, 0.013183976523578167, 0.02791900746524334, 0.006852343678474426, -0.008320307359099388, -0.030666593462228775, 0.0009666402474977076, -0.03698160871863365, 0.028162745758891106, 0.01433618925511837, -0.014513452537357807, 0.0021576297003775835, 0.02304426021873951, 0.013693608343601227, -0.03680434450507164, -0.021393494680523872, 0.01448021549731493, -0.017715275287628174, 0.004902444779872894, 0.013084265403449535, -0.004340187180787325, 0.02853943035006523, -0.013804398477077484, -0.007921464741230011, 0.0017435530899092555, -0.035851553082466125, 0.03164154291152954, -0.019332805648446083, -0.028916114941239357, -0.008431097492575645, 0.02805195562541485, -0.029780276119709015, -0.007240107748657465, -0.02129378356039524, 0.006309474352747202, -0.003777929348871112, -0.021903127431869507, 0.027232112362980843, 0.035541340708732605, 0.004553457256406546, 0.013904109597206116, -0.013427713885903358, 0.004445437341928482, -0.024927685037255287, 0.0006965903448872268, 0.028783168643712997, 0.005833078641444445, 0.024307264015078545, 0.007605713792145252, 0.02348741888999939, -0.007578016724437475, -0.0037363830488175154, -0.006298395339399576, 0.004583924543112516, -0.03678218647837639, 0.0045589967630803585, 0.023443102836608887, -0.0062430002726614475, -0.005317906383424997, 0.02421863190829754, 0.025769688189029694, 0.02971380203962326, 0.005448084324598312, -0.004919063299894333, -0.02528221346437931, -0.014358347281813622, 0.03682650253176689, 0.0030300987418740988, 0.00819843914359808, 0.025636740028858185, 0.04108082503080368, 0.044847678393125534, -0.025902634486556053, -0.0008163817110471427, -0.01905583031475544, -0.03130917251110077, 0.01329476572573185, 0.054597172886133194, 0.01980920135974884, -0.052691590040922165, 0.011499972082674503, 0.02526005543768406, -0.013904109597206116, 0.03986214101314545, -0.0088631771504879, -0.030223434790968895, -0.026102056726813316, 0.0019152772147208452, -0.0030356382485479116, 0.0031381186563521624, 0.011954210698604584, 0.03609528765082359, -0.016984064131975174, 0.0001455846067983657, -0.018490804359316826, 0.016928669065237045, -0.02364252507686615, 0.0010234200162813067, 0.020163727924227715, 0.02718779630959034, -0.0054425448179244995, -0.0028722232673317194, -0.027896849438548088, 0.0003159237967338413, -0.019399279728531837, -0.00900720339268446, 0.018468646332621574, 0.0428534634411335, -0.010253587737679482, 0.0005612191744148731, 0.006625224836170673, 0.006852343678474426, 0.014956611208617687, 0.003420632565394044, -0.0025343147572129965, -0.0310432780534029, 0.016153139993548393, -0.007024067919701338, -0.02676679566502571, -0.014502373524010181, -0.01767095923423767, -0.0036255933810025454, 0.01603127270936966, 0.006697238422930241, -0.0020523795392364264, -0.005761065054684877, 0.007472766097635031, -0.013704687356948853, 0.02155967801809311, -0.011250695213675499, 0.029824592173099518, -0.020196964964270592, -0.011632920242846012, 0.03075522556900978, 0.011555367149412632, -0.007544779684394598, -0.030843855813145638, -0.017294274643063545, 0.04258756712079048, -0.060048025101423264, 0.010995878838002682, 0.03485444560647011, -0.027542322874069214, -0.013870872557163239, -0.022268733009696007, 0.029447905719280243, -0.006979751866310835, 0.025481633841991425, -0.009256480261683464, 0.013039949350059032, 0.012762974947690964, -0.013483108021318913, 0.014291873201727867, -0.009334033355116844, 0.0011986062163487077, -0.004373424220830202, 0.004711332730948925, -0.019587621092796326, 0.005921710282564163, -0.02030775509774685, -0.014435899443924427, -0.0006612761062569916, -0.04079277440905571, 0.00956669170409441, -0.0317523330450058, -0.028118429705500603, -0.020684439688920975, 0.01060811523348093, 0.029669485986232758, 0.039396822452545166, 0.03724750131368637, 0.032284121960401535, 0.020584728568792343, 0.014624242670834064, -0.009793810546398163, -0.0114667359739542, -0.01655198261141777, 0.007317660842090845, -0.029492221772670746, 0.007611253298819065, 0.023686841130256653, 0.013859793543815613, -0.021471045911312103, -0.022999944165349007, -0.01572106033563614, -0.04037177190184593, 0.013693608343601227, -0.015133875422179699, 0.043695464730262756, 0.0020399156492203474, 0.008962888270616531, -0.008741308934986591, -0.0058386181481182575, -0.0457783117890358, -0.055217593908309937, -0.003271066350862384, -0.021969599649310112, -0.017615564167499542, -0.048082735389471054, -0.02040746621787548, 0.006220842711627483, 0.015887245535850525, 0.033480651676654816, 0.014723952859640121, 0.013505266048014164, 0.024728264659643173, -0.03722534328699112, 0.010619194246828556, 0.03928603231906891, -0.028893956914544106, 0.018878567963838577, -0.05145074427127838, -0.024573158472776413, 0.02304426021873951, -0.0031021120958030224, 0.003813935909420252, 0.013028870336711407, -0.002598018851131201, -0.014768268913030624, -0.024418054148554802, -0.015831850469112396, 0.00011433845065766945, -0.015211427584290504, 0.01926633156836033, -0.01730535365641117, -0.016950827091932297, 0.007533700671046972, 0.01639687828719616, 0.006182066164910793, 0.012641106732189655, -0.002312735188752413, 0.02776390314102173, 0.023155050352215767, 0.01515603344887495, 0.02554810792207718, 0.009555612690746784, 0.0023556663654744625, 0.0025329298805445433, 0.010563799180090427, -0.02239060215651989, -0.015133875422179699, 0.004993846639990807, 0.010801997035741806, 0.018080880865454674, -0.007361976429820061, 0.010419772937893867, -0.021393494680523872, 0.003692067228257656, -0.058319706469774246, 0.026279320940375328, -0.011499972082674503, 0.015532718040049076, 0.007384134456515312, 0.017549090087413788, -0.006968672852963209, -0.0029248485807329416, -0.040416087955236435, -0.03159722685813904, -0.0030827238224446774, -0.02439589612185955, -0.02424078993499279, -0.016175298020243645, 0.002679726341739297, 0.0005812997696921229, -0.030090486630797386, -0.0206955187022686, -0.031419962644577026, 0.024152157828211784, -0.022434918209910393, 0.008514189161360264, -0.035851553082466125, -0.020496096462011337, 0.004456516355276108, -0.011200840584933758, -0.07857206463813782, 0.00031834733090363443, -0.03979566693305969, -0.03999508544802666, -0.02291131392121315, -0.05273590609431267, -0.023155050352215767, -0.02215794287621975, 0.007001909893006086, -0.004126917105168104, 0.006580909248441458, -0.036006659269332886, 0.008951809257268906, -0.004680865444242954, -0.004437128081917763, 0.04429372772574425, -0.041169457137584686, 0.00506309000775218, 0.03221764788031578, -0.00045319917262531817, 0.00027783983387053013, 0.016518747434020042, 0.026988374069333076, 0.01915554143488407, 0.006392566487193108, -0.013206133618950844, -0.022013915702700615, 0.05065305903553963, 0.03297102078795433, 0.018446488305926323, -0.017837144434452057, 0.009350651875138283, -0.055660754442214966, 0.019953228533267975, 0.03935250639915466, -0.0030965725891292095, -0.001704776776023209, 0.052957482635974884, 0.014535610564053059, -0.02426294796168804, -0.015100638382136822, -0.046620313078165054, 0.00046531681437045336, -0.012984554283320904, -0.039551928639411926, -0.019100146368145943, 0.02599126659333706, -0.013604977168142796, 0.005370531231164932, -0.0066640013828873634, 0.019620858132839203, 0.0008496186346746981, -0.01315073948353529, 0.03268296644091606, -0.002455377019941807, 0.0006384257576428354, -0.01247492153197527, 0.010824155062437057, 0.06385919451713562, -0.007445069029927254, 0.021748021245002747, -0.059649184346199036, -0.01774851232767105, 0.0077885170467197895, 0.01574321836233139, 0.01970949023962021, 0.006741554010659456, 0.015865087509155273, 0.0007997632492333651, -0.008336925879120827, -0.009223244152963161, -0.04311935603618622, 0.021171914413571358, -0.05007695034146309, 0.003043947508558631, -0.046177152544260025, 0.0016743096057325602, -0.04591125622391701, -0.023841947317123413, 0.0012830833438783884, -0.01299563329666853, 0.0003493338299449533, -0.004232167266309261, -0.021648310124874115, 0.012807291001081467, -0.022246574983000755, -0.018125196918845177, 0.0708611011505127, -0.007101620547473431, 0.007999017834663391, 0.010885089635848999, 0.002296116901561618, -0.024174315854907036, 0.01463532168418169, 0.016275009140372276, -0.014834742993116379, -0.01441374234855175, -0.005345603916794062, 0.03644981607794762, -0.02179233729839325, 0.014646400697529316, 0.002078691963106394, 0.00034292880445718765, 0.008913032710552216, 0.0010435006115585566, 0.028317851945757866, 0.029625169932842255, 0.018069801852107048, 0.028184903785586357, 0.025038475170731544, -0.04068198427557945, -0.032727282494306564, -0.005342834163457155, 0.02986890636384487, -0.043518200516700745, 0.006270697806030512, 0.011577525176107883, -0.00595494732260704, -0.009755033999681473, 0.0162417721003294, 0.019587621092796326, -0.005478551611304283, 0.03651629015803337, 0.0024290645960718393, -0.034366969019174576, -0.004190620966255665, -0.007157015614211559, 0.04094787687063217, -0.010840773582458496, 0.004326338414102793, -0.020584728568792343, -0.009500217624008656, -0.004769497085362673, 0.00595494732260704, 0.002550933277234435, -0.002449837513267994, 0.034344810992479324, -0.03771281987428665, -0.017560169100761414, -0.00882440060377121, -0.02113867737352848, -0.011001418344676495, 0.006198684684932232, 0.002321044448763132, -0.014757189899682999, -0.014945532195270061, -0.003777929348871112, -0.035253286361694336, -0.016407957300543785, 0.04389488697052002, 0.0007208255701698363, -0.015931561589241028, 0.03385733813047409, -0.04564536362886429, 0.005428696051239967, -0.028406482189893723, 0.00908475648611784, 0.0032766058575361967, -0.0184797253459692, 0.03133133053779602, -0.012685422785580158, -0.02588047832250595, -0.01271865889430046, -0.03780145198106766, 0.013826556503772736, 0.004268173594027758, 0.02494984306395054, -0.005977105349302292, -0.01411460991948843, -0.010181575082242489, -0.017781749367713928, 0.04555673152208328, 0.03407891467213631, -0.00279605551622808, -0.022224416956305504, 0.0007471381686627865, -0.019908912479877472, -0.038200292736291885, -0.03204038739204407, -0.01359389815479517, 0.006830186117440462, -0.022169021889567375, -0.004204469732940197, -0.012264421209692955, -0.02381978929042816, 0.004680865444242954, -0.020429624244570732, 0.016950827091932297, -0.017504775896668434, 0.049146316945552826, 0.01263002771884203, 0.005971565842628479]}>, <Node element_id='4:c66ce224-058b-46a0-b519-9d2e36be1920:1212' labels=frozenset({'__Entity__', 'Concept'}) properties={'human_readable_id': 33, 'name': 'LIFE-IN-DEATH', 'description': "Life-in-Death is a personification of a nightmarish figure that represents the struggle between life and death, described as thickening man's blood with cold.", 'id': 'eae4259b19a741ab9f9f6af18c4a0470', 'description_embedding': [0.0026752171106636524, 0.07085242867469788, -0.010720080696046352, 0.031404584646224976, -0.013883592560887337, -0.004825316835194826, 0.02702433615922928, -0.0024879039265215397, 0.04713249206542969, 0.03665575757622719, 0.01863526552915573, -0.008658033795654774, -0.04861818999052048, -0.02366870827972889, 0.04948911443352699, 0.015932831913232803, -0.008376263082027435, 0.04380247741937637, -0.020979082211852074, 0.05963284894824028, 0.009042265824973583, -0.028817420825362206, 0.025372138246893883, 0.04923295974731445, -0.0055937813594937325, 0.021555431187152863, 0.026460794731974602, 0.014856981113553047, 0.049514733254909515, -0.006269389763474464, 0.034837059676647186, -0.02062046527862549, 0.03181443363428116, -0.03788530081510544, 0.05246051400899887, -0.023489398881793022, 0.013025473803281784, 0.011853565461933613, -0.017482567578554153, -0.04131777584552765, -0.03404298052191734, -0.04247047379612923, -0.01066884957253933, 0.0626554787158966, -0.04605663940310478, 0.005382453557103872, 0.0320705883204937, -0.02128646709024906, 0.0406261570751667, 0.01648356392979622, -0.029278498142957687, 0.0045883734710514545, 0.008600398898124695, 0.065217025578022, -0.017687492072582245, 0.01919880509376526, -0.022938666865229607, 0.04234239459037781, -0.018084531649947166, -0.008331436663866043, 0.0065447562374174595, -0.014639249071478844, -0.011610218323767185, 0.026332717388868332, -0.024975096806883812, -0.03911484405398369, -0.02961149998009205, 0.03814145550131798, -0.03265974298119545, 0.016790950670838356, 0.02516721375286579, -0.016970258206129074, -0.06475594639778137, 0.005814714822918177, -0.05968407914042473, 0.05317774415016174, -0.021952470764517784, 0.009093496948480606, -0.033274512737989426, 0.002702433615922928, 0.0037430627271533012, 0.01746976003050804, 0.002202931558713317, 0.06398747861385345, -0.0008128914050757885, -0.03027750365436077, -0.036066602915525436, -0.0010462324135005474, 0.0083122244104743, -0.0008901380933821201, 0.03847445920109749, -0.00010666447633411735, -0.029124805703759193, -0.018865805119276047, -0.014536787755787373, 0.01681656576693058, 0.0045083253644406796, 0.028766188770532608, -0.04016508162021637, 0.02548740804195404, 0.027382953092455864, 0.04987334832549095, 0.05166643112897873, 0.014088516123592854, 0.028048954904079437, -0.01707272045314312, -0.029790809378027916, -0.012032873928546906, -0.04216308891773224, -0.022989897057414055, -0.057788532227277756, -0.0014200584264472127, 0.0004346627392806113, 0.08053508400917053, 0.002242955844849348, 0.004642806481570005, 0.03931976854801178, 0.031148429960012436, 0.03435036540031433, 0.0191347673535347, 0.04024192690849304, 0.031788814812898636, -0.03376120701432228, 0.0011542977299541235, 0.0002969795314129442, -0.032890282571315765, 0.02509036660194397, -0.0029505835846066475, -0.022554434835910797, -0.023591861128807068, -0.0017914829077199101, 0.007678241468966007, -0.020453965291380882, -0.07828091830015182, -0.03373559191823006, 0.012359471060335636, 0.011693468317389488, 0.027434183284640312, -0.030764196068048477, -0.03791091591119766, -0.02961149998009205, -0.021388929337263107, 0.021171197295188904, 0.009977230802178383, 0.06583179533481598, 0.03532375395298004, 0.004549950361251831, 0.004469902254641056, 0.048720650374889374, -0.036706987768411636, 0.016393909230828285, 0.007703857030719519, 0.0028929486870765686, 0.002868934301659465, 0.005110288970172405, 0.0014680874301120639, 0.0034388788044452667, 0.005913975182920694, -0.02088942751288414, -0.038704998791217804, 0.00709548918530345, -0.04111285135149956, 0.03496513515710831, 0.01164223812520504, -0.0206332728266716, 0.01739291474223137, -0.027459800243377686, 0.00023674311523791403, -0.017495375126600266, 0.028689343482255936, -0.024616479873657227, 0.00959940254688263, -0.041497085243463516, -0.011142735369503498, -0.050155118107795715, -0.005657820031046867, 0.01745695248246193, 0.012654049322009087, -0.02922726795077324, -0.02393767051398754, 0.015023482032120228, 0.00959940254688263, -0.02219581790268421, 0.021517006680369377, -0.034375980496406555, -0.01092500425875187, 0.0027760781813412905, 0.007300413213670254, 0.012231393717229366, 0.010624022223055363, 0.0175850298255682, 0.018519995734095573, 0.003550946479663253, -0.0221830103546381, 0.037347376346588135, 0.037475455552339554, 0.05614914372563362, 0.025641100481152534, 0.012500356882810593, 0.025448983535170555, 0.056558988988399506, 0.009881173260509968, -0.001970791257917881, -0.013486552983522415, -0.004806105513125658, 0.052229974418878555, -0.004617190919816494, -0.010054077953100204, 0.029201652854681015, -0.018673688173294067, 0.021504199132323265, -0.04818272590637207, 0.014536787755787373, -0.055380675941705704, 0.02258004993200302, 0.011238793842494488, 0.010143731720745564, -0.015305251814424992, -0.011155543848872185, 0.046594567596912384, 0.03921730816364288, -0.011027466505765915, 0.007844742387533188, 0.032634127885103226, 0.0021517006680369377, 0.001352017279714346, 0.01465205755084753, -0.0382695347070694, 0.0032179453410208225, 0.006115696858614683, 0.03096912056207657, -0.007620606571435928, -0.023643091320991516, -0.044596560299396515, -0.02587164007127285, -0.011949623934924603, -0.03847445920109749, 0.016022484749555588, 0.014921019785106182, 0.001120677450671792, -0.04236800968647003, 0.05148712545633316, 0.012686069123446941, 0.00726199010387063, -0.04882311448454857, 0.005062260199338198, -0.0031747191678732634, 0.005366444122046232, -0.02205493301153183, -0.012429913505911827, -0.008491532877087593, -0.0240017082542181, 0.036963142454624176, -0.04623594880104065, 0.007793511264026165, 0.024629289284348488, 0.027562260627746582, -0.010124520398676395, 0.005321616772562265, -0.030431196093559265, 0.005638608708977699, 0.0175850298255682, -0.009983635507524014, -0.003653408493846655, -0.0042233532294631, -0.05153835564851761, 0.0023294081911444664, 0.0141909783706069, 0.034171056002378464, 0.03281343728303909, 0.0033908498007804155, -0.02722926065325737, 0.002137291943654418, -0.0008072879863902926, -0.01686779595911503, -0.014588018879294395, 0.036245912313461304, 0.0461847186088562, -0.021107159554958344, 0.027587877586483955, -0.028638111427426338, -0.013038281351327896, 0.029842039570212364, -0.06736872345209122, -0.0876561850309372, 0.035477444529533386, -0.03163512423634529, -0.005661021918058395, -0.0003384045558050275, -0.016726911067962646, -0.013191974721848965, 0.03286466747522354, 0.003131492994725704, 0.01771310716867447, 0.03125089034438133, -0.030636118724942207, -0.037091221660375595, -0.0071531240828335285, -0.007614202797412872, 0.01274370402097702, 0.04223993420600891, -0.036886297166347504, -0.0461847186088562, 0.02529529109597206, -0.028919881209731102, 0.06716380268335342, 0.0013872386189177632, -0.010233386419713497, 0.009829942137002945, -0.026166217401623726, 0.03253166377544403, 0.045365024358034134, 0.0246420968323946, 0.03675822168588638, -0.010425502434372902, -0.016662873327732086, 0.023502206429839134, 0.01732887513935566, 0.00726199010387063, -0.05251174420118332, 0.044212326407432556, -0.0011863171821460128, -0.009375267662107944, 0.0007072275038808584, 0.01673971861600876, 0.013473744504153728, 0.01533086784183979, 0.01006048172712326, -0.04085669666528702, -0.044212326407432556, 0.019121959805488586, 0.016201794147491455, -0.044212326407432556, 0.00941369030624628, 0.026281487196683884, -0.05256297439336777, -0.007703857030719519, 0.01798206940293312, -0.01592002436518669, -0.05599544942378998, -0.019544614478945732, 0.02702433615922928, -0.03401736170053482, 0.02535932883620262, -0.001732247183099389, 0.015100328251719475, 0.014344671741127968, -0.01991603896021843, 0.018993882462382317, 0.010675253346562386, -0.02943219244480133, 0.02489825151860714, -0.03798776492476463, -0.03839761018753052, -0.044852714985609055, 0.03422228619456291, 0.01338409073650837, 0.013396898284554482, 0.04177885502576828, 0.02023623324930668, -0.05860823020339012, -0.04610787332057953, -0.009131920523941517, 0.0542023666203022, -0.03867937996983528, -0.04403301700949669, 0.016265831887722015, 0.011725488118827343, 0.02147858403623104, -0.027280490845441818, 0.019941654056310654, 0.0031394977122545242, -0.007838338613510132, 0.028048954904079437, -0.0028273090720176697, 0.0308410432189703, 0.0034612922463566065, -0.029893269762396812, -0.017508184537291527, -0.013665861450135708, -0.014216593466699123, 0.05271666869521141, 0.022349510341882706, 0.0008733279537409544, 0.02845880389213562, 0.06613918393850327, -0.02205493301153183, 0.018391918390989304, -0.04275224357843399, 0.01082894578576088, 0.05768607184290886, -0.07126227766275406, -0.0028401168528944254, 0.02825387939810753, -0.04267539829015732, -0.029201652854681015, 0.010912196710705757, 0.1060481071472168, 0.01641952618956566, -0.004296997562050819, 0.004335420671850443, -0.0025919668842107058, -0.04428917169570923, -0.009458517655730247, -0.013781130313873291, 0.02017219364643097, 0.08258432149887085, 0.008030454628169537, -0.024731749668717384, -0.009029458276927471, 0.014549595303833485, -0.028766188770532608, 0.015394906513392925, 0.03491390496492386, 0.016406718641519547, -0.035016369074583054, 0.07310659438371658, -0.0006828127661719918, 0.023591861128807068, 0.03668137267231941, -0.015817562118172646, -0.021119967103004456, 0.008555571548640728, -0.061477161943912506, 0.022272663190960884, 0.009900384582579136, 0.026742566376924515, 0.0382695347070694, 0.043290168046951294, -0.01825103349983692, 0.026089370250701904, 0.015612637624144554, -0.008401879109442234, 0.022003700956702232, -0.033991746604442596, -0.019416537135839462, 0.0013776327250525355, -0.03829514980316162, -0.05148712545633316, -0.004476306028664112, -0.009272805415093899, -0.0021677103359252214, 0.01932688243687153, 0.015561406500637531, -0.02807457186281681, 0.024936674162745476, -0.012974242679774761, 0.006749680265784264, 0.0255514457821846, -0.046594567596912384, 0.038064610213041306, -0.027972109615802765, 0.007787107490003109, 0.021939663216471672, -0.029124805703759193, -0.004934182856231928, 0.004892557393759489, -0.01270528044551611, 0.0001848117244662717, -0.04805465042591095, -0.04503202065825462, -0.06429486721754074, 0.007255585864186287, -0.0003293991321697831, 0.008882169611752033, 0.033530671149492264, 0.01965988427400589, 0.025448983535170555, -0.014626441523432732, -0.02547459863126278, 0.0013704284792765975, 0.0009365661535412073, -0.008254589512944221, -0.00100700871553272, -0.004553152248263359, 0.03739860653877258, -0.009259997867047787, 0.018763342872262, -0.03775722533464432, 0.04836203530430794, 0.03588729351758957, -0.025525830686092377, -0.038448844105005264, -0.022669704630970955, 0.014549595303833485, -0.006186139304190874, -0.016393909230828285, -0.028535649180412292, 0.012897396460175514, -0.010790523141622543, -0.015522983856499195, -0.015369290485978127, 0.015254020690917969, -0.012935820035636425, 0.011168351396918297, 0.006768891587853432, 0.029713962227106094, -0.018276648595929146, 0.03824391961097717, -0.01577913761138916, 0.014639249071478844, 0.01804610900580883, 0.057327453047037125, 0.029637115076184273, 0.027203643694519997, -0.003474100027233362, 0.04037000238895416, 0.0039063612930476665, -0.06823965162038803, -0.0009917995193973184, -0.022541627287864685, 0.07443860173225403, 0.02366870827972889, 0.01656041108071804, -0.015958447009325027, -0.027178028598427773, -0.01624021679162979, 0.01545894518494606, -0.009003843180835247, 0.033274512737989426, 0.0024991107638925314, -0.0048381248489022255, 0.00789597351104021, -0.0006419880664907396, 0.04262416809797287, -0.022541627287864685, 0.0011911200126633048, 0.05128220096230507, -0.013332859613001347, 0.011834354139864445, -0.0157150998711586, -0.02973957732319832, 0.04649210348725319, 0.037552300840616226, -0.013793938793241978, 0.021568238735198975, -0.015061904676258564, -0.028894266113638878, -0.01393482368439436, -0.0314301997423172, 0.01132204383611679, 0.02328447625041008, 0.04813149571418762, 0.015753522515296936, -0.002920165192335844, 0.0382695347070694, -0.006519140675663948, -0.04915611445903778, -0.008068877272307873, 0.005382453557103872, 0.0430852435529232, 0.0019227622542530298, -0.003762274282053113, -0.01611213944852352, -0.02277216501533985, 0.008722072467207909, 0.018289456143975258, -0.02290024235844612, -0.004934182856231928, 0.0015433329390361905, -0.006435890682041645, -0.04121531546115875, -0.03519567474722862, -0.01577913761138916, -0.028561266139149666, -0.018648073077201843, -0.020646080374717712, -0.0010854561114683747, 0.01486978866159916, 0.024014517664909363, -0.016931835561990738, -0.020146578550338745, -0.010393482632935047, -0.04741426184773445, 0.010380675084888935, -0.0002583561872597784, 0.0028257081285119057, -0.006583179347217083, -0.012173758819699287, 0.017930839210748672, 0.005731464829295874, -0.027434183284640312, -0.04313647747039795, 0.013166358694434166, -0.030559273436665535, 0.032173048704862595, -0.013793938793241978, -0.024398749694228172, -0.017303260043263435, -0.034760214388370514, 0.03109719790518284, 0.0029009536374360323, 0.034504055976867676, 0.012359471060335636, -0.0013288032496348023, -0.014933827333152294, 0.010438309982419014, -0.015369290485978127, 0.026537641882896423, -0.03788530081510544, -0.026460794731974602, 0.013768322765827179, -0.01940372958779335, 0.00853636022657156, 0.008971823379397392, 0.0034004554618149996, 0.0240529403090477, 0.0005371246952563524, -0.0005907571176066995, -0.022913051769137383, -0.0030610503163188696, -0.006368649657815695, -0.019839193671941757, -0.03020065650343895, 0.001563345082104206, -0.0055137332528829575, -0.008907784707844257, -0.01919880509376526, 0.009266401641070843, -0.026460794731974602, -0.016470756381750107, -0.015010673552751541, 0.0097274798899889, -0.014319055713713169, 0.017098335549235344, -0.007114700973033905, -0.05491960048675537, -0.045954179018735886, -0.019160382449626923, 0.02185000851750374, -0.0061925435438752174, 0.028945498168468475, -0.012916607782244682, -0.0289711132645607, 0.0025087164249271154, 0.003358830465003848, -0.039934542030096054, -0.023233244195580482, -0.04546748474240303, -0.03524690493941307, -0.009426497854292393, -0.01516436692327261, 0.03306959196925163, -0.03248043358325958, 0.03368436172604561, 0.0033428207971155643, 0.006563968025147915, 0.008260994218289852, 0.012762915343046188, -0.02010815590620041, 0.007249182090163231, 0.0141909783706069, -0.013665861450135708, -0.005769887939095497, -0.002191724954172969, -0.013870785012841225, -0.030149424448609352, 0.0031555076129734516, 0.033274512737989426, 0.011411698535084724, 0.014075708575546741, -0.011776719242334366, 0.007345240097492933, 0.0166244488209486, -0.0005559360724873841, 0.0002737655013334006, 0.023182014003396034, -0.026038140058517456, -0.034171056002378464, 0.015497367829084396, 0.07039134949445724, 0.025192828848958015, 0.02884303592145443, -0.005199943203479052, 0.01604810170829296, 0.009279209189116955, -0.00224775867536664, -0.014408710412681103, -0.0425473190844059, -0.005087875761091709, 0.005065462086349726, 0.02871495857834816, 0.00570905115455389, -0.03493952006101608, 0.027254875749349594, 0.007870357483625412, 0.07039134949445724, 0.022106163203716278, -0.0028945498634129763, 0.007927992381155491, -0.02232389524579048, 0.010143731720745564, -0.023527823388576508, -0.03939661383628845, 0.023297283798456192, -0.005785897374153137, 0.006698449142277241, 0.036373987793922424, -0.0007480522035621107, 0.018289456143975258, 0.0022829801309853792, -0.002343816915526986, -0.028100186958909035, -0.005212751217186451, -0.012442721985280514, -0.02322043664753437, 0.027382953092455864, 0.047234952449798584, 0.008331436663866043, -0.05169204622507095, -0.014575210399925709, 0.004325815010815859, 0.008894977159798145, 0.04282908886671066, -0.01857122592628002, 0.03163512423634529, -0.02619183249771595, 0.035605523735284805, 0.011533372104167938, 0.04615910351276398, 0.003755870508030057, 0.0026720152236521244, -0.04946349933743477, -0.04505763575434685, 0.017444144934415817, 0.02205493301153183, 0.04134339094161987, -0.020582042634487152, 0.023643091320991516, -0.015574214980006218, -0.01422940194606781, -0.008529956452548504, 0.009842749685049057, -0.025525830686092377, -0.007319624535739422, -0.024629289284348488, -0.012276221066713333, 0.011795930564403534, -0.004287391901016235, 0.014395901933312416, 0.005155116319656372, -0.003531734924763441, -0.04085669666528702, 0.006685641594231129, 0.03435036540031433, -0.006487121339887381, 0.007383663207292557, -1.4558801012753975e-05, 0.0049085672944784164, -0.020518003031611443, -0.012173758819699287, 0.01022057794034481, -0.0023374129086732864, -0.049770887941122055, 0.004617190919816494, -0.011552583426237106, 0.005891561508178711, 0.009714672341942787, -0.006445496343076229, -0.014088516123592854, -0.0058723497204482555, 0.006993027403950691, 0.037347376346588135, -0.02509036660194397, -0.002202931558713317, -0.025961292907595634, -0.018276648595929146, 0.005741070490330458, 0.03622029349207878, -0.027972109615802765, -0.031455814838409424, -0.042777858674526215, 0.0068137189373373985, 0.000786875665653497, 0.016265831887722015, -0.03985769301652908, -0.023041129112243652, 0.04751672223210335, 0.012993454933166504, -0.005846734158694744, 0.015958447009325027, -0.01836630329489708, 0.018186993896961212, -0.028740573674440384, 0.03865376487374306, -0.06987904012203217, -0.006461505778133869, -0.022016508504748344, -0.05297282338142395, 0.008427494205534458, -0.007844742387533188, 0.0034100613556802273, -0.010009250603616238, -0.011994450353085995, -0.025577060878276825, 0.012410702183842659, 9.91224169410998e-06, -0.07612922042608261, 0.023707130923867226, -0.0664977952837944, 0.046978797763586044, 0.02833072654902935, -0.0034548884723335505, 0.05835207551717758, -0.029022343456745148, -0.021184004843235016, -0.015945639461278915, 0.004610787145793438, 0.0024238652549684048, 0.01566386967897415, 0.013358475640416145, 0.016470756381750107, -0.0012135335709899664, 0.02658887207508087, -0.003960794303566217, -0.045749254524707794, -0.00671125715598464, -0.03732176125049591, 8.67024136823602e-05, -0.042444858700037, 0.037603531032800674, -0.0166244488209486, -0.012698876671493053, -0.03901238366961479, -0.021132774651050568, -0.007210758980363607, -0.007351643871515989, 0.02270812727510929, 0.010636829771101475, -0.023002704605460167, -0.0077614919282495975, 0.03181443363428116, -0.01533086784183979, 0.005840330384671688, -0.03368436172604561, -0.014767326414585114, 0.011366871185600758, -0.03422228619456291, 0.00671125715598464, 0.006343034561723471, 0.04390494152903557, -0.0018299061339348555, 0.007204355206340551, -0.03875622898340225, 0.034427210688591, 0.023246051743626595, -0.00853636022657156, -0.013768322765827179, 0.020223425701260567, 0.04825957119464874, -0.036373987793922424, -0.04567240923643112, 0.04185570031404495, -0.02031307853758335, 0.02264408767223358, -0.02380959317088127, -0.009881173260509968, 0.02606375515460968, -0.0012695674085989594, -0.027382953092455864, 0.006439092569053173, -0.04241924360394478, 0.00030598495504818857, 0.042982783168554306, 0.004329016897827387, -0.022400740534067154, -0.0045851715840399265, 0.024949481710791588, 0.013243205845355988, 0.02424505539238453, 0.017431337386369705, 0.019941654056310654, -0.021132774651050568, -0.016342679038643837, -0.01629144884645939, 0.0056258006952703, -0.019185997545719147, -0.006519140675663948, -0.007409278769046068, 0.006595987360924482, -0.01791803166270256, 0.03983207792043686, 0.012205778621137142, -0.02424505539238453, -0.03248043358325958, -0.027536645531654358, 0.02062046527862549, 0.010950619354844093, -0.0837114006280899, 0.02017219364643097, 0.003086665878072381, -0.009477728977799416, -0.012340259738266468, -0.005344030447304249, 0.026435179635882378, 0.013435321860015392, 0.011956027708947659, 0.015369290485978127, -0.003759072395041585, 0.005456098355352879, -0.009221574291586876, 0.002697630785405636, -0.02017219364643097, -0.013742707669734955, 0.022874627262353897, 0.03181443363428116, 0.016726911067962646, 0.01351216807961464, -0.004694037605077028, 0.016457948833703995, 0.011097908951342106, -0.018545610830187798, -0.040318772196769714, 0.04492956027388573, 0.03409421071410179, 0.009522556327283382, -0.03898676857352257, 0.0013792337849736214, 0.023950478062033653, -0.046927567571401596, 0.025692330673336983, 0.03199373930692673, 0.022106163203716278, 0.016086524352431297, -0.01986480876803398, 0.028868651017546654, -0.013729900121688843, -0.025128791108727455, 0.00989398080855608, 0.014421517960727215, -0.013960438780486584, 0.014690480194985867, -0.04054931178689003, -0.012308239936828613, -0.005446492228657007, 0.026665719226002693, 0.0051903375424444675, 0.0034548884723335505, 0.025192828848958015, 0.007025046739727259, 0.00811370462179184, -0.02140173688530922, 0.0058755516074597836, 0.042521703988313675, -0.02606375515460968, -0.012596414424479008, 0.019698306918144226, 0.0027808810118585825, 0.012852569110691547, 0.02275935746729374, -0.009202362969517708, 0.021568238735198975, 0.027331722900271416, -0.009689057245850563, 0.013051089830696583, -0.014703287743031979, 0.03401736170053482, 0.00802405085414648, 0.0006387861212715507, -0.00888857338577509, 0.0013792337849736214, -0.007863953709602356, 0.013320052064955235, -0.03555429354310036, 0.017431337386369705, -0.04218870401382446, -0.0399857722222805, -0.0009053472895175219, -0.02477017417550087, 0.0031010746024549007, -0.03988330811262131, 0.02451401948928833, 0.03445282578468323, 0.019173189997673035, 0.03224989399313927, -0.009016650728881359, -0.021888431161642075, 0.010694464668631554, 0.02243916504085064, -4.452691791811958e-05, 0.021452968940138817, 0.004072862211614847, -0.03486267477273941, -0.010758503340184689, -0.020082540810108185, -0.029509037733078003, 0.011302832514047623, 0.0016441938932985067, -0.017162375152111053, 0.010559983551502228, -0.003125089220702648, -0.02689625881612301, 0.0013063896913081408, 0.016445141285657883, 0.041164085268974304, 0.031071582809090614, -0.014792942441999912, -0.0017482568509876728, 0.024718942120671272, 0.016214601695537567, -0.0005935588269494474, -0.014882596209645271, 0.006150918081402779, 0.006256582215428352, -0.011981642805039883, 0.04897680506110191, 0.03575921803712845, -0.018315071240067482, -0.0038551304023712873, 0.009836345911026001, 0.018481571227312088, 0.015407714061439037, 0.08114985376596451, 0.010137327946722507, 0.020133771002292633, 0.03125089034438133, -0.008472321555018425, 0.022682512179017067, -0.02491105906665325, 0.04920734465122223, 0.021158389747142792, -0.004761278163641691, -0.024462787434458733, 0.03952469304203987, 0.03350505232810974, -0.021901238709688187, 0.0087925149127841, 0.006948200054466724, 0.02450121007859707, -0.019109152257442474, 0.012622030451893806, 0.038448844105005264, 0.029534652829170227, 0.024885443970561028, -0.02625587210059166, 0.014805749990046024, 0.009682653471827507, 0.03522128984332085, -0.006871353834867477, 0.003041838761419058, -0.06854704022407532, 0.0020028105936944485, -0.00527038611471653, -0.026460794731974602, 0.003365234239026904, -0.022490395233035088, 0.0068201227113604546, 0.011078696697950363, 0.029380960389971733, -0.017994878813624382, 0.024526827037334442, 0.021978085860610008, -0.0023806390818208456, 0.05399744212627411, 0.0271011833101511, 0.01609933190047741, -0.016906220465898514, 0.019250037148594856, 0.03199373930692673, -0.034171056002378464, 0.010182155296206474, -0.01927565224468708, 0.01648356392979622, -0.008696457371115685, 0.002015618374571204, 0.010054077953100204, 0.026947489008307457, 0.0010718479752540588, 0.004441084805876017, 0.012103316374123096, 0.0018331081373617053, 0.04516009986400604, -0.013845168985426426, 0.005462502129375935, 0.0246420968323946, 0.02534652128815651, -0.015087520703673363, -0.006589583586901426, -0.00996442325413227, 0.016086524352431297, 0.014280633069574833, 0.0007376458961516619, 0.008523552678525448, -0.016714103519916534, -0.03588729351758957, -0.007287605199962854, 0.021427351981401443, 0.01959584653377533, -0.019506191834807396, -0.013563399203121662, -0.002851323690265417, 0.00805606972426176, -0.019711114466190338, -0.024718942120671272, -0.0013368080835789442, -0.03078981302678585, 0.028535649180412292, -0.01809733919799328, 0.015369290485978127, -0.03650206699967384, 0.027203643694519997, -0.008715668693184853, -0.025269675999879837, -0.009375267662107944, -0.03926853835582733, -0.031071582809090614, -0.02109435200691223, -0.0041689202189445496, 0.03222427889704704, 0.009279209189116955, 0.02081258036196232, -0.016150562092661858, 0.010329443961381912, 0.013781130313873291, 0.019903231412172318, 0.030098194256424904, -0.0020012096501886845, -0.006589583586901426, -0.0015025082975625992, 0.03319766744971275, 0.000253953505307436, 0.018468763679265976, 0.0358104482293129, 0.031455814838409424, 0.013627437874674797, 0.04508325457572937, 0.02418101765215397, 0.002694428898394108, 0.029509037733078003, 0.014242209494113922, -0.006240572314709425, 0.01791803166270256, -0.035989757627248764, -0.01958303712308407, 0.028100186958909035, -0.024847019463777542, -0.010066885501146317, 0.015932831913232803, 0.02529529109597206, 0.02355343848466873, 0.025820408016443253, -0.014690480194985867, 0.005974811967462301, -0.02548740804195404, -0.014690480194985867, 0.030098194256424904, -0.020287463441491127, 0.08652910590171814, -0.0011534972582012415, -0.03053365834057331, 0.034632135182619095, 0.041804470121860504, -0.010470328852534294, 0.017751531675457954, 0.009138324297964573, -0.04029315710067749, 0.011053081601858139, 0.00038863494410179555, -0.0002741657372098416, 0.03058488853275776, 0.012468337081372738, 0.020261848345398903, 0.008094493299722672, 0.009324036538600922, -0.008549167774617672, 0.0168934129178524, 0.0221830103546381, 0.0036854278296232224, -0.01577913761138916, 0.0022221433464437723, -0.015753522515296936, -0.015382098034024239, -0.01739291474223137, 0.004421873018145561, -0.005923580843955278, 0.014280633069574833, 0.0020508398301899433, -0.015305251814424992, -0.00911911204457283, 0.021747546270489693, 0.005564963910728693, 0.03660452738404274, 0.02179877646267414, -0.005667426157742739, -0.03409421071410179, -0.021324891597032547, 0.033914901316165924, -0.003086665878072381, -0.0219140462577343, -0.03209620341658592, -0.004066457971930504, 0.030559273436665535, 0.030379964038729668, -0.012141739949584007, -0.013281628489494324, 0.04300839826464653, 0.029329730197787285, -0.015804754570126534, 0.016201794147491455, 0.017623452469706535, 0.022656895220279694, 0.007876761257648468, -0.027767185121774673, 0.001847516861744225, 0.0035605523735284805, -0.013115128502249718, 0.03155827522277832, -0.0197239238768816, 0.024731749668717384, -0.05230681970715523, -0.0049245767295360565, 0.029457807540893555, 0.0008805322577245533, 0.01804610900580883, -0.00111987697891891, -0.006583179347217083, -0.03058488853275776, 0.001518517965450883, 0.019711114466190338, 0.0025599475484341383, -0.0181741863489151, -0.011085101403295994, -0.012993454933166504, 0.0009990038815885782, -0.016009677201509476, -0.0068905651569366455, -0.025205636397004128, -0.016842180863022804, 0.017482567578554153, 0.011200370267033577, -0.03988330811262131, -0.0019275652011856437, -0.04385371133685112, 0.00200601271353662, -0.03665575757622719, -0.004553152248263359, -0.0007636616355739534, 0.0004038441111333668, 0.01688060350716114, 0.028228264302015305, 0.032377973198890686, 0.023771170526742935, -0.018455956131219864, -0.004421873018145561, -0.00032199465204030275, -0.013589014299213886, 0.00022533621813636273, -0.0024542836472392082, -0.002201330615207553, 0.0004282588779460639, 0.003640600712969899, 0.011859969235956669, 0.024091362953186035, 0.00892059225589037, 0.002371033187955618, -0.03427352011203766, -0.028433186933398247, -0.0388074591755867, 0.007934396155178547, 0.023463783785700798, 0.027075566351413727, -0.003554148366674781, 0.006112494971603155, -0.006099687423557043, 0.01145012117922306, -0.010252597741782665, -0.00028937493334524333, -0.016457948833703995, -0.025500215590000153, -0.022157393395900726, 0.0259228702634573, 0.019800769165158272, 0.021632276475429535, -0.013998862355947495, 0.015945639461278915, 0.03445282578468323, -0.0181741863489151, -0.014267824590206146, 0.016457948833703995, -0.013704284094274044, 0.0037398608401417732, -0.050027042627334595, -0.022810589522123337, -0.007831934839487076, -0.04111285135149956, 0.00895901583135128, 0.011334852315485477, -0.031455814838409424, -0.033863671123981476, 0.0015689483843743801, -0.017316067591309547, 0.01303187757730484, 0.0021517006680369377, -0.012564395554363728, 0.009970827028155327, 0.0009861961007118225, 0.005654618144035339, 0.0006563967908732593, 0.035656753927469254, -0.011597410775721073, -0.006986623629927635, 0.00023474190675187856, -0.034427210688591, -0.01882738061249256, 0.029765192419290543, -0.015189982019364834, -0.012532375752925873, 0.018340686336159706, -0.004806105513125658, -0.02328447625041008, -0.030303118750452995, 0.014203785918653011, -0.010226982645690441, -0.0004938985803164542, -0.019442152231931686, 0.022477587684988976, -0.048464495688676834, -0.0032371568959206343, -0.04597979411482811, 0.02799772471189499, -0.002471894258633256, -0.012346663512289524, 0.01900669001042843, 0.012180162593722343, -0.016573218628764153, 0.022554434835910797, 0.013832361437380314, -0.003720649052411318, -0.017764339223504066, -0.0004930980503559113, 0.0007324427715502679, -0.021888431161642075, 0.007255585864186287, -0.004104881547391415, -0.022477587684988976, 0.01648356392979622, -0.031788814812898636, 0.04357193782925606, -0.0017274442361667752, -0.03770599141716957, 0.015100328251719475, -0.031711969524621964, -0.008517148904502392, -0.021504199132323265, -0.05322897806763649, 0.01686779595911503, -0.017879609018564224, 0.004764480050653219, -0.004437882918864489, 0.01024619396775961, -0.0046492102555930614, -0.04111285135149956, 0.028766188770532608, 0.003197132609784603, -0.026819411665201187, -0.013537783175706863, -0.046261563897132874, -0.035656753927469254, -0.006221360992640257, 0.006845738273113966, -0.02658887207508087, 0.0034548884723335505, 0.0033460226841270924, -0.03150704503059387, 0.004533940926194191, 0.012468337081372738, 0.013653052970767021, -0.006150918081402779, -0.01435747928917408, -0.006621602922677994, 0.009195959195494652, 0.033274512737989426, 0.03250604867935181, 0.03078981302678585, 0.005296001210808754, 0.01796926185488701, -0.04410986602306366, 0.009913192130625248, 0.046978797763586044, -0.015151559375226498, 0.009234381839632988, -0.0019739933777600527, -0.014024477452039719, 0.008555571548640728, 0.0071467203088104725, -0.034119825810194016, 0.013396898284554482, -0.01503628958016634, -0.014716096222400665, 0.01040629018098116, 6.639013008680195e-05, -0.021312084048986435, -0.00520314509049058, -0.016342679038643837, -0.013409705832600594, 0.022106163203716278, 0.010105309076607227, 0.023438168689608574, 0.03250604867935181, 0.029995732009410858, 0.0015617440221831203, 0.03919169306755066, 0.029534652829170227, -0.0019803971517831087, 0.038704998791217804, -0.027562260627746582, 0.02986765466630459, -0.029585884883999825, 0.04513448476791382, 0.02258004993200302, -0.034427210688591, 0.00921517051756382, 0.006583179347217083, -0.0240529403090477, 0.002079657046124339, -0.011655045673251152, -0.024949481710791588, -0.03732176125049591, 0.0055361464619636536, 0.023425361141562462, -0.02579479292035103, -0.027280490845441818, -0.014536787755787373, -0.028638111427426338, 0.009689057245850563, 0.02076135016977787, 0.002342215972021222, -0.0039095631800591946, 0.0018539207521826029, -0.023591861128807068, 0.0049245767295360565, 0.02794649451971054, -0.030636118724942207, 0.0030754590407013893, 0.005897965282201767, -0.007364451885223389, 0.004722855053842068, 0.02968834713101387, -0.010009250603616238, 0.002415860304608941, 0.016675680875778198, 0.00575387803837657, -0.009913192130625248, 0.004175323992967606, 0.009003843180835247, 0.01914757490158081, -0.015881599858403206, -0.028305109590291977, -0.005219154991209507, -0.004277785774320364, 0.03716806694865227, 0.015433329157531261, 0.05353636294603348, 0.004434680566191673, -0.0031090795528143644, -0.06234809011220932, -0.005507329013198614, -0.029918886721134186, -0.019877616316080093, -0.011258005164563656, -0.0002641596947796643, 0.020786965265870094, -0.0051743281073868275, -0.013115128502249718, 0.016675680875778198, 0.02735733799636364, -0.0083122244104743, 0.021555431187152863, -0.016919028013944626, 0.02204212360084057, 0.004617190919816494, 0.06373132765293121, -0.027331722900271416, 0.007671837694942951, -0.03757791593670845, 0.009266401641070843, 0.023694323375821114, 0.013589014299213886, 0.016137754544615746, -0.020069731399416924, -0.006189341656863689, -0.014370286837220192, 0.007633414585143328, -0.0024959088768810034, -0.003592571709305048, 0.021645084023475647, 0.009272805415093899, -0.014306248165667057, -0.004796499386429787, 0.020210618153214455, 0.021517006680369377, -0.011488544754683971, -0.021388929337263107, -0.0215938538312912, -0.006413477007299662, -0.03227550908923149, -0.015574214980006218, 0.0030066175386309624, -0.013435321860015392, -0.014370286837220192, -0.00805606972426176, 0.016675680875778198, -0.047952186316251755, 0.038781844079494476, -0.020133771002292633, -0.060298848897218704, 0.023886438459157944, -0.001118276035413146, 0.020095348358154297, 0.018776150420308113, 0.016393909230828285, 0.01811014674603939, -0.007697453256696463, 0.017367297783493996, -0.04889995977282524, 0.047900956124067307, 0.05748114734888077, -0.011283621191978455, -0.011277217417955399, 0.010214174166321754, -0.0400882326066494, 0.027178028598427773, -0.0240529403090477, 0.03265974298119545, 0.02917603775858879, -0.0017114345682784915, 0.0018283051904290915, 0.0048509323969483376, -0.03555429354310036, -0.007691049017012119, -0.008164935745298862, -0.02917603775858879, 0.011514159850776196, 0.03552867844700813, 0.015241213142871857, 0.02010815590620041]}>, <Node element_id='4:c66ce224-058b-46a0-b519-9d2e36be1920:4699' labels=frozenset({'__Entity__', 'Concept'}) properties={'human_readable_id': 3520, 'name': 'SIMPLE LIFE', 'description': 'The simple life is presented as the only path to true happiness, contrasting with the complexities of modern existence.', 'id': '521d5dd328f947fb977efca21c523dd0', 'description_embedding': [0.017167828977108, 0.009630582295358181, -5.985244206385687e-05, 0.0285139549523592, 0.007723045535385609, -0.013489007949829102, 0.007840718142688274, 0.040826234966516495, 0.015309838578104973, 0.03834891691803932, -0.04744068160653114, -0.014541869051754475, -0.023447835817933083, -0.01606542058289051, 0.004025645088404417, 0.003914165776222944, -0.017911024391651154, 0.00010983431275235489, 0.07749057561159134, 0.04335310310125351, 0.012293701060116291, -0.011606245301663876, 0.04649930074810982, 0.04436880350112915, 0.0169944167137146, -0.017539426684379578, -0.03200697526335716, 0.033642008900642395, 0.06584717333316803, -0.010187978856265545, 0.005224049091339111, -0.024042392149567604, 0.04176761955022812, -0.026284366846084595, 0.01938503049314022, 0.03027285262942314, -0.0022094598971307278, -0.012708652764558792, 0.020475050434470177, -0.03993440046906471, 0.02504570782184601, 0.02088380977511406, 0.006360519677400589, 0.02204814925789833, 0.030644450336694717, 0.01783670485019684, -0.00439724326133728, 0.037828680127859116, 0.012349440716207027, 0.02289043925702572, -0.04293195903301239, -0.03527703881263733, 0.002968139247968793, 0.05058687925338745, -0.06346894055604935, -0.021020062267780304, 0.009525296278297901, 0.017527040094137192, -0.005896022077649832, 0.056235168129205704, 0.033369503915309906, 0.00613446393981576, 0.03735798969864845, 0.04810955747961998, -0.03805163875222206, -0.006428645923733711, -0.024785587564110756, -0.000703713740222156, 0.0005202372558414936, 0.03440997749567032, -0.000898028549272567, -0.056383807212114334, 0.011922103352844715, 0.01065247692167759, -0.031065594404935837, -0.0007431960548274219, -0.029752615839242935, 0.01705634966492653, 0.006205686833709478, -0.03158583119511604, -0.011247033253312111, 0.04320446401834488, -0.019050592556595802, 0.0351283997297287, -0.0035270845983177423, -0.007308094762265682, -0.013067863881587982, 0.010076499544084072, -0.012411373667418957, 0.04538450390100479, 0.03582204878330231, -0.02809281088411808, -0.020809490233659744, -0.0044251130893826485, 0.05281646549701691, -0.015941554680466652, 0.007803558371961117, 0.04030599817633629, -0.03200697526335716, 0.03166015073657036, 0.03252721577882767, -0.012231768108904362, -0.007933617569506168, 0.007778785191476345, 0.02223394811153412, 0.03691207244992256, 0.009624389000236988, -0.007636339403688908, -0.02866259403526783, 0.006242847070097923, -0.06976133584976196, 0.010658670216798782, 0.02103244885802269, 0.09988555312156677, 0.01483914814889431, 0.06069434434175491, -0.022729413583874702, 0.0256650373339653, 0.06232937425374985, 0.0035828242544084787, 0.022122468799352646, -0.01801011711359024, -0.030074667185544968, 0.005202372558414936, -0.015384158119559288, 0.012782972306013107, 0.051379621028900146, 0.004747164901345968, -0.056383807212114334, -0.02878645993769169, 0.0014337491011247039, 0.03958757594227791, -0.04275854676961899, -0.011420446448028088, 0.03297313302755356, -0.0364413782954216, -0.004663555417209864, 0.0060075013898313046, 0.04392288625240326, 0.0362679660320282, 0.013588101603090763, -0.060793437063694, -0.01214506197720766, 0.009828767739236355, 0.005487264133989811, 0.009927860461175442, 0.011835397221148014, 0.024389216676354408, 0.009655355475842953, 0.007933617569506168, -0.016560886055231094, -0.017279308289289474, -0.027869852259755135, 0.04275854676961899, 0.023831820115447044, -0.014715282246470451, 0.02591276913881302, -0.02105722203850746, -0.0059672449715435505, -0.01462857611477375, 0.05011618882417679, -0.02289043925702572, 0.012015002779662609, 0.007326674647629261, 0.002311649499461055, -0.013427074998617172, -0.026457779109477997, -0.03497976064682007, -0.0547487773001194, -0.010126045905053616, -0.014913467690348625, 0.01583007536828518, 0.003898682538419962, -0.0043322136625647545, 0.009704901836812496, -0.011878750286996365, -0.0207599438726902, -0.058712489902973175, -0.04122260957956314, 0.014380843378603458, 0.01663520559668541, 0.023323969915509224, -0.0007722271257080138, 0.0013253663200885057, 0.056681085377931595, -0.03138764575123787, -0.025392532348632812, 0.007828331552445889, -0.016895323991775513, -0.01773761212825775, -0.049050938338041306, 0.009810187853872776, -0.027597347274422646, -0.00665779784321785, 0.023200104013085365, 0.031313326209783554, -0.016238834708929062, 0.038844380527734756, -0.002760663628578186, 0.021428819745779037, -0.01726692169904709, -0.017452720552682877, -0.008658234030008316, -0.03978576138615608, -0.029603976756334305, 0.03502930700778961, -1.2271686500753276e-05, 0.015334611758589745, -0.004576848819851875, 0.005106376018375158, 0.0207599438726902, -0.06574808061122894, 0.021465979516506195, 0.004939157050102949, 0.02028925158083439, 0.008844032883644104, 0.012993544340133667, -0.010268491692841053, -0.031189460307359695, 0.06034751981496811, 0.014219817705452442, -0.0040287417359650135, 0.027993718162178993, 0.008583914488554, -0.006694957613945007, 0.03386496752500534, -0.015991101041436195, -0.03309699892997742, -0.015322225168347359, 0.000650296569801867, 0.03510362654924393, 0.027547800913453102, -0.039265524595975876, -0.016350314021110535, 0.028191903606057167, -0.016461793333292007, 0.00870778039097786, -0.03473202884197235, -0.030718769878149033, 0.050785064697265625, -0.027151430025696754, -0.011841590516269207, -0.06163572520017624, 0.011469992808997631, -0.010522417724132538, -0.014442776329815388, -0.026408232748508453, -0.021020062267780304, -0.04652407392859459, -0.02521912008523941, 0.02282850630581379, 0.06891904771327972, -0.022432133555412292, -0.02749825455248356, 0.0170811228454113, -0.03805163875222206, 0.00013286565081216395, -0.041346475481987, 0.007574406452476978, -0.004632588941603899, 0.0015312934992834926, 0.02940579131245613, 0.016238834708929062, -0.034335657954216, 0.013699580915272236, 0.0038677160628139973, 0.022803733125329018, -0.02485990710556507, 0.013129796832799911, 0.0003768236201722175, 0.007215194869786501, -0.018988659605383873, -0.01235563401132822, 0.0009282209211960435, -0.03857187554240227, -0.023138171061873436, -0.0005272047128528357, 0.026457779109477997, -0.011247033253312111, 0.041569434106349945, -0.0056699663400650024, -0.015359384939074516, 0.04682135209441185, 0.029009418562054634, 0.008658234030008316, -0.02160223200917244, 0.047886598855257034, 0.05147871375083923, 0.016288381069898605, -0.03386496752500534, -0.031461965292692184, -0.029579203575849533, -0.029207605868577957, -0.011086007580161095, 0.025442078709602356, -0.033790647983551025, 0.0025361564476042986, 0.0017604456515982747, 0.02621004730463028, -0.0007950649596750736, 0.024661721661686897, -0.024587402120232582, -0.010924981907010078, -0.005264305509626865, 0.03750662878155708, 0.026036635041236877, -0.0020360476337373257, -0.020747557282447815, 0.0020747557282447815, 0.012498080730438232, -0.046028610318899155, -0.030148986726999283, -0.015458477661013603, -0.0046759420074522495, 0.020338797941803932, -0.011853977106511593, -0.06411304324865341, 0.014430389739573002, 0.02238258719444275, -0.029133284464478493, 0.01555757038295269, -0.01680861786007881, 0.04028122499585152, 0.030471038073301315, 0.03582204878330231, -0.0013826542999595404, 1.807185799407307e-05, -0.0030269755516201258, -0.019186845049262047, -0.00933330412954092, -0.005555390380322933, -0.03270062804222107, -0.03562386333942413, 0.010181785561144352, -0.0005047539598308504, -0.07268457114696503, -0.006273813545703888, 0.0292571522295475, -0.044715628027915955, 0.02046266384422779, 0.03864619508385658, 0.001957082888111472, -0.01627599447965622, -0.005412944592535496, -0.027597347274422646, 0.0042857639491558075, 0.04154466092586517, -0.0292571522295475, 0.03173447027802467, 0.049794137477874756, 0.0028179516084492207, -0.020128225907683372, -0.017688065767288208, -0.01353855524212122, -0.010163205675780773, 0.005796929355710745, 0.0409996472299099, -0.01116032712161541, -0.000519850174896419, 0.0521228164434433, 0.003437281819060445, -0.04211444407701492, 0.015656663104891777, 0.01893911324441433, -0.0254916250705719, -0.008887385949492455, 0.0013896217569708824, -0.07169364392757416, -0.011414253152906895, 0.002063917461782694, -0.004078288562595844, -0.04464130848646164, 0.014269364066421986, -0.028439635410904884, 0.020152999088168144, 0.007351447828114033, 0.005261208862066269, -0.016709525138139725, -0.042857639491558075, -0.05950523167848587, -0.020896196365356445, 0.02342306263744831, -0.03235379979014397, -0.03438520431518555, 0.0699099749326706, 0.06445986777544022, -0.013662421144545078, -0.04367515444755554, -0.00917227752506733, -0.045310184359550476, 0.04320446401834488, 0.0053045619279146194, -0.00031605185358785093, 0.034236565232276917, -0.002855111612007022, -0.06089252978563309, 0.002178493421524763, 0.0524696409702301, 0.03324563801288605, -0.02485990710556507, 0.06772993505001068, 0.051379621028900146, -0.015446091070771217, 0.01241756696254015, -0.005016573239117861, -0.00503515312448144, -0.016536112874746323, -0.018381714820861816, -0.045136772096157074, 0.026878923177719116, 0.03790299966931343, 0.0046356855891644955, 0.008144190534949303, -0.009407623670995235, -0.01705634966492653, -0.06976133584976196, 0.0033319955691695213, 0.011618631891906261, 0.030495811253786087, -0.030718769878149033, 0.012411373667418957, -0.018257848918437958, 0.05370829999446869, -0.0026708608493208885, -0.0207599438726902, 0.0902239978313446, 0.0687704086303711, -0.030049894005060196, 0.000305020046653226, -0.010280879214406013, 0.016771458089351654, 0.03252721577882767, -0.00571331987157464, 0.006843597162514925, -0.0038770060054957867, 0.022097695618867874, -0.0387205146253109, -0.04699476435780525, 0.03141241893172264, -0.0023348743561655283, 0.0008964802254922688, -0.005474877543747425, -0.07714375108480453, -0.041123516857624054, 0.0015591634437441826, -0.013092637062072754, -0.012721039354801178, 0.0037964931689202785, -0.047044310718774796, -0.00031779371784068644, -0.0364413782954216, 0.012646719813346863, 0.020487437024712563, -0.01747749373316765, -0.007630146108567715, 0.04944730922579765, 0.0027513736858963966, -0.04672225937247276, -0.0008763520163483918, -0.0067940508015453815, 0.030768316239118576, 0.001412846613675356, 0.023881366476416588, 0.028018491342663765, 0.04218876361846924, 0.04119783639907837, 0.07347731292247772, -0.03287404030561447, -0.005292175337672234, -0.009413816966116428, -0.032329026609659195, -0.044145844876766205, 0.01843126304447651, -0.004121641628444195, -0.03827459737658501, 0.028588274493813515, -0.0039358423091471195, 0.05891067534685135, 0.02967829629778862, 0.01672191172838211, 0.009258984588086605, -0.015210745856165886, 0.009184664115309715, 0.0047750347293913364, -0.01138328667730093, -0.02536775916814804, 0.033220864832401276, -0.04288241267204285, -0.052915558218955994, 0.02022731862962246, -0.007716852240264416, -0.04065282270312309, 0.01440561655908823, -0.01135231927037239, -0.02752302773296833, -0.048307742923498154, -0.015272678807377815, 0.035524770617485046, 0.009655355475842953, -0.018084436655044556, -0.008144190534949303, 0.05464968457818031, 0.0037748166359961033, 0.024451149627566338, 0.06941451132297516, 0.006750697270035744, 0.016585659235715866, -0.019967200234532356, 0.038695741444826126, 0.004109255038201809, -0.06505443155765533, -0.004369373433291912, 0.03475680202245712, -0.04714340344071388, 0.030421491712331772, -0.010559577494859695, -0.018468422815203667, 0.04166852682828903, 0.02429012395441532, -0.001460844767279923, 0.029455337673425674, -0.0061004008166491985, -0.01957082934677601, -0.0030378138180822134, 0.03213084116578102, 0.01490108110010624, 0.023732727393507957, -0.002557832980528474, -0.03956280276179314, -0.006471998989582062, -0.0330226793885231, 0.00022082986833993345, 0.022952372208237648, 0.027275295928120613, -0.013811060227453709, 0.03691207244992256, -0.01283251866698265, 0.00994644034653902, 0.01383583340793848, -0.01033661887049675, 0.004604719113558531, -0.019769014790654182, 0.04526063799858093, 0.06609490513801575, -0.007140875328332186, 0.015780529007315636, -0.004710005130618811, -0.00860249437391758, -8.806098776403815e-05, 0.012398987077176571, 0.03354291617870331, -0.016461793333292007, 0.020809490233659744, -0.02839008904993534, 0.031461965292692184, 0.015334611758589745, 0.01568143628537655, 0.056681085377931595, 0.007617759518325329, 0.07833286374807358, -0.011841590516269207, -0.037655267864465714, 0.016858164221048355, 0.01680861786007881, 0.014764828607439995, -0.012547627091407776, -0.006471998989582062, 0.018183529376983643, 0.038695741444826126, -0.004654265474528074, 0.012386600486934185, -0.00298826745711267, -0.03490544110536575, 0.020351184532046318, -0.023955686017870903, 0.02648255228996277, -0.012919224798679352, -0.0012185318628326058, -0.02492184191942215, -0.0008407405111938715, -0.012547627091407776, -0.04320446401834488, -0.05157780647277832, -0.027300069108605385, -0.04137124866247177, -0.013290822505950928, -0.01735362783074379, -0.024872295558452606, -0.03919120505452156, 0.04194103181362152, -0.005992018152028322, 0.0066825710237026215, 0.030594903975725174, 0.02124302089214325, -0.040256451815366745, -0.0021862350404262543, -0.006688764318823814, -0.000509398989379406, -0.027151430025696754, -0.035351358354091644, -0.007729238830506802, -0.008441468700766563, 0.024302510544657707, -0.03834891691803932, -0.025330599397420883, -0.01729169487953186, -0.026755057275295258, 0.017242148518562317, 0.044145844876766205, -0.02564026415348053, 0.002732793800532818, 0.005323141813278198, 0.04820865020155907, -0.020846650004386902, -0.00734525453299284, -0.0011271806433796883, 0.0009746706346049905, 0.03802686557173729, 0.005586356855928898, -0.03012421354651451, -0.017192602157592773, -0.008026516996324062, 0.02779553271830082, 0.009977406822144985, -0.010627703741192818, -0.010280879214406013, -0.05088415741920471, 0.0019539862405508757, 0.01957082934677601, -0.014120724983513355, 0.013067863881587982, 0.03329518437385559, -0.008955512195825577, -0.00466665206477046, 0.04174284636974335, 0.03096650168299675, 0.005604936741292477, 0.010534804314374924, -0.007673499174416065, -0.01794818416237831, 0.022295881062746048, 0.0109931081533432, 0.03864619508385658, -0.03502930700778961, -0.018480809405446053, 0.03639183193445206, 0.025020934641361237, -0.024797974154353142, 0.006004404742270708, -0.04496335983276367, 0.030446264892816544, 0.0035270845983177423, 0.05633426085114479, -0.0182206891477108, 0.0007404864882119, 0.03009944036602974, -0.004295053891837597, 0.05271737277507782, 0.006942689884454012, 0.015012560412287712, -0.030916955322027206, 0.016895323991775513, -0.013798673637211323, 0.004874127451330423, 0.02836531586945057, 0.031610604375600815, 0.023385902866721153, 0.0011310514528304338, -0.05128052830696106, 0.001749607385136187, 0.019224004819989204, -0.004793614614754915, 0.010132239200174809, 0.05559106543660164, 0.02821667678654194, 0.0044715628027915955, 0.035648636519908905, -0.030148986726999283, 0.009810187853872776, -0.005697836633771658, -0.0028984646778553724, 0.0007029395783320069, -0.041148290038108826, -0.007400994189083576, -0.008237089961767197, -0.011110780760645866, -0.0068869502283632755, 0.04969504103064537, -0.009822574444115162, -0.00241229054518044, -0.036515697836875916, -0.0011372447479516268, 0.02318771742284298, -0.009494329802691936, -0.010057919658720493, -0.02187473699450493, 0.032031748443841934, 0.05435240641236305, 0.006571091711521149, -0.029207605868577957, -0.022333040833473206, 0.008825452998280525, -0.08289113640785217, -0.019645148888230324, -0.024277737364172935, -0.017093509435653687, -0.00013170440797694027, 0.006546318531036377, 0.02381943352520466, -0.02141643315553665, 0.02500854805111885, -0.031139913946390152, 0.028885552659630775, -0.018332168459892273, 0.0007505505927838385, 0.001909084850922227, -0.023311583325266838, 0.01612735353410244, -0.0005484941648319364, 0.044418349862098694, -0.016536112874746323, 0.026110954582691193, -0.04278331995010376, -0.017390787601470947, -0.0053912680596113205, 0.003839846234768629, 0.010559577494859695, 0.01570620946586132, 0.006626831367611885, 0.061437539756298065, 0.014566642232239246, -0.02593754231929779, 0.0142569774761796, 0.025615490972995758, 0.03656524419784546, -0.02722574956715107, -0.01860467530786991, 0.0182206891477108, 0.04332832992076874, 0.02965352311730385, -0.01073918305337429, 0.0071594552136957645, -0.0354752242565155, 0.043006278574466705, -0.009934053756296635, 0.02145359292626381, -0.007667305879294872, -0.02767166681587696, -0.07099999487400055, -0.03698639199137688, 0.011389479972422123, 0.009989793412387371, -0.012008809484541416, 0.011754884384572506, 0.02725052274763584, -0.05227145552635193, -0.019991973415017128, 0.016647592186927795, 0.04637543484568596, -0.008738746866583824, 0.017192602157592773, 0.02046266384422779, -0.005731899756938219, 0.009023638442158699, 0.008769713342189789, 0.02908373810350895, 0.02088380977511406, 0.0040597086772322655, -0.001299818977713585, -0.04057850316166878, -0.03713503107428551, 0.020091066136956215, -0.007946004159748554, 0.004648072179406881, 0.03411269932985306, -0.010850662365555763, -0.0008113223593682051, 0.009927860461175442, -0.04263468086719513, -0.019917653873562813, 0.017551813274621964, -0.03195742890238762, -0.015545183792710304, -0.008868806064128876, 0.012721039354801178, -0.007301901467144489, 0.012454727664589882, -0.020301638171076775, 0.024661721661686897, -0.009872120805084705, 0.024401603266596794, 0.016709525138139725, -0.015000173822045326, -0.0006955850403755903, -0.024587402120232582, 0.031040821224451065, 0.007103715557605028, -0.08318841457366943, 0.030148986726999283, -0.02109438180923462, -0.008168963715434074, -0.014455162920057774, -0.01779954507946968, 0.03289881348609924, -0.014071178622543812, -0.002994460752233863, -0.03971144184470177, -0.013154570013284683, -0.019434576854109764, -0.018109209835529327, 0.020920969545841217, 0.0042393142357468605, -0.015656663104891777, 0.02462456189095974, 0.007209001574665308, -0.03673865646123886, -0.028885552659630775, 0.0033536721020936966, -0.022543612867593765, 0.009711095131933689, 0.012516660615801811, -0.0009235759498551488, 0.03698639199137688, -0.015532797202467918, -0.01636270061135292, 0.018840020522475243, -0.04263468086719513, 0.004731681663542986, -0.007487700320780277, -0.02895987220108509, -0.026903696358203888, -0.01758897304534912, 0.0003919198061339557, 0.01052861101925373, 0.011915910057723522, 0.01779954507946968, 0.00039908080361783504, 0.0041061583906412125, 0.0035549544263631105, 0.02079710364341736, 0.04109874367713928, 0.05167689919471741, -0.04699476435780525, 0.007078942377120256, 0.04526063799858093, -0.020896196365356445, 0.0041526081040501595, -0.0027699535712599754, -0.0072585479356348515, 0.013241276144981384, -0.03611932694911957, -0.006713537499308586, -0.007419574074447155, 0.011086007580161095, -0.0199300404638052, -0.0023952589835971594, -0.0018905049655586481, -0.01274581253528595, -0.010225138626992702, 0.0338154211640358, -0.014244590885937214, -0.00041766068898141384, 0.026755057275295258, 0.05891067534685135, 0.015842461958527565, -0.025058094412088394, -0.008683007210493088, 0.002588799688965082, -0.002621314488351345, -0.006571091711521149, -0.026433005928993225, 0.029727842658758163, -0.003660240676254034, -0.00885641947388649, 0.026903696358203888, -0.008119417354464531, 0.017762385308742523, 0.004809097852557898, -0.06480669230222702, 0.022184401750564575, -0.009345690719783306, 0.023918526247143745, 0.005697836633771658, -0.029356244951486588, -0.0326758548617363, -0.015470864251255989, 0.004487046040594578, 0.02551639825105667, -0.021825190633535385, -0.006868370342999697, -0.0028102099895477295, -0.028315769508481026, -0.007933617569506168, 0.0034806348849087954, -0.014071178622543812, -3.3893804356921464e-05, -0.024587402120232582, 0.04107396677136421, 0.03673865646123886, 0.02318771742284298, 0.0009506716160103679, -0.0060539511032402515, 0.01069582998752594, 0.01762613281607628, 0.004542785696685314, 0.012844905257225037, 0.02375750057399273, -0.017601359635591507, 0.013736740685999393, 0.01932309754192829, 0.012919224798679352, -0.005580163560807705, -0.026160500943660736, -0.006527738645672798, 0.030594903975725174, 0.014727668836712837, -0.0043260203674435616, 0.025962315499782562, 0.02420341782271862, -0.008881192654371262, 0.04204012453556061, -0.019496509805321693, -0.036664336919784546, -0.00727712782099843, 0.015000173822045326, -0.018592288717627525, -9.357689850730821e-05, 0.0206732377409935, 0.02995080128312111, 0.00090422184439376, 0.015297451987862587, 0.007109908852726221, -0.04417061805725098, -0.01783670485019684, -0.0065896715968847275, -0.006478192284703255, 0.0016892226412892342, 0.004319827072322369, 0.0031678732484579086, -0.02414148487150669, 0.01410833839327097, -0.02090858295559883, 0.02982693538069725, 0.07213956117630005, 0.002015919191762805, -0.013365142047405243, -0.014269364066421986, 0.010119852609932423, -0.009698708541691303, 0.0033877354580909014, 0.010918788611888885, -0.0013153022155165672, -0.01144521962851286, 0.006818823982030153, -0.007209001574665308, -0.010386165231466293, 0.007549633271992207, -0.03344382345676422, 0.00795839074999094, -0.022283494472503662, 0.0013060122728347778, 0.052221909165382385, -0.006843597162514925, 0.00723377475515008, -0.02141643315553665, 0.011061234399676323, -0.031189460307359695, -0.01295638456940651, -0.008292829617857933, -0.008069870062172413, -0.042733773589134216, 0.0029247861821204424, -0.018802860751748085, 0.019050592556595802, 0.02043789066374302, -0.001749607385136187, -0.015780529007315636, -0.0036695306189358234, 0.05628471449017525, -0.05549197271466255, 0.016957256942987442, 0.008218510076403618, -0.015532797202467918, 0.0016241930425167084, -0.027919398620724678, -0.024698881432414055, -0.038398463279008865, 0.005750479642301798, -0.0012967223301529884, 4.586913200910203e-05, 0.011977843008935452, -0.01648656651377678, -0.0094385901466012, -0.00691791670396924, 0.0031926464289426804, 0.008652040734887123, 0.0030858118552714586, -0.008831646293401718, 0.0008531271596439183, 0.017960570752620697, -0.0017604456515982747, 0.03054535761475563, 0.03750662878155708, 0.023051464930176735, 0.0009816380916163325, -0.009389043785631657, -0.03488066792488098, 0.011668178252875805, 0.008447661995887756, -0.007698272354900837, 0.022518839687108994, 0.015359384939074516, 0.01169914472848177, 0.007933617569506168, -0.0005620419979095459, -0.04275854676961899, -0.026854149997234344, 0.019248778000473976, -0.02441398985683918, 0.028910325840115547, 0.006558705121278763, -0.0016969643766060472, -0.00822470337152481, -0.02722574956715107, 0.03693684563040733, 0.013340368866920471, -0.02234542742371559, -0.01971946842968464, -0.0032545793801546097, 0.007970777340233326, 0.027597347274422646, -0.020214932039380074, -0.0009204793022945523, 0.011922103352844715, 0.024599788710474968, 0.025417305529117584, 0.003384638810530305, 0.047316815704107285, -0.025739356875419617, -0.011178907006978989, -0.022097695618867874, 0.021565072238445282, 0.0033505754545331, -0.0016381279565393925, -0.0349549874663353, 0.009810187853872776, 0.0017449624137952924, -0.0015901299193501472, 0.0256650373339653, -0.024822747334837914, -0.018468422815203667, 0.028117584064602852, -0.01612735353410244, 0.01268387958407402, -0.00906699150800705, -0.010912595316767693, 0.011092200875282288, 0.0003108262608293444, 0.020487437024712563, 0.027423935011029243, -0.025466851890087128, -0.02507048100233078, 0.017576586455106735, 0.01621405966579914, 0.020933356136083603, -0.009389043785631657, -0.01798534393310547, 0.01374912727624178, -0.014418003149330616, -0.009178470820188522, 0.03428611159324646, -0.005930085200816393, 0.007760205306112766, 0.02139165997505188, -0.014529482461512089, -0.013773900456726551, 0.003833652939647436, -0.0031802598387002945, 0.0030114923138171434, -0.010311845690011978, 0.012114095501601696, 0.030446264892816544, 0.02168893814086914, -0.01779954507946968, -0.04337787628173828, -0.025813676416873932, 0.02500854805111885, -0.015805302187800407, -0.007877877913415432, 0.010813502594828606, 0.002833434846252203, -0.0013044639490544796, -0.004068998619914055, -0.011860170401632786, 0.004951543640345335, -0.021218247711658478, 0.005700933281332254, -0.016647592186927795, 0.01591678149998188, 0.017663292586803436, 0.03562386333942413, 0.0043260203674435616, 0.02285327948629856, -0.018914340063929558, -0.007400994189083576, -0.02139165997505188, -0.025206733494997025, 0.02103244885802269, -0.017601359635591507, 0.009952633641660213, -0.019855720922350883, -0.022778959944844246, 0.023918526247143745, 0.0010714409872889519, -0.009612002409994602, 0.02722574956715107, 0.009853540919721127, -0.004710005130618811, -0.009017445147037506, 0.002426225459203124, -0.02593754231929779, -0.04095010086894035, 0.034657709300518036, 0.03282449394464493, 0.012485694140195847, -0.007097522262483835, 0.008683007210493088, -0.0055708736181259155, 0.0001828959066187963, 0.03673865646123886, -0.009704901836812496, -0.005409847944974899, -0.020301638171076775, -0.02725052274763584, -0.007029396016150713, -0.014418003149330616, -0.004706908483058214, -0.026755057275295258, -0.0010606027208268642, 0.0010776342824101448, 0.046301115304231644, -0.01978140138089657, -0.0003381154965609312, -0.010392358526587486, -0.02648255228996277, -0.0008957060636021197, 0.033072225749492645, 0.05896022170782089, -0.017452720552682877, -0.017886251211166382, 0.02384420670568943, 0.016412246972322464, 0.00839192233979702, -0.010627703741192818, 0.021069608628749847, 0.025466851890087128, 0.03468248248100281, -0.007654919289052486, -0.028836006298661232, 0.0256650373339653, 0.017960570752620697, -0.018827633932232857, -0.0018936016131192446, -0.026135727763175964, 0.005521327257156372, -0.013290822505950928, -0.0015111652901396155, 0.020561758428812027, -0.016684751957654953, 0.010813502594828606, -0.010460484772920609, -0.05113188922405243, -0.0068002440966665745, 0.017031576484441757, 0.008732553571462631, -0.02749825455248356, 0.013934926129877567, 0.005462490953505039, 0.023014305159449577, -0.022518839687108994, 0.013910152949392796, 0.0273743886500597, 0.019397417083382607, 4.088546120328829e-05, -0.04035554453730583, -0.04226308315992355, -0.011977843008935452, -0.0027482770383358, -0.012770585715770721, -0.04008303955197334, -0.015966327860951424, 0.02139165997505188, 0.03827459737658501, -0.025144800543785095, -0.00560184009373188, 0.04612770304083824, 0.03557431697845459, -0.07555826753377914, 0.016350314021110535, 0.0012510466622188687, -0.00039540353463962674, -0.0003646305704023689, 0.002440160373225808, 0.027845079079270363, -0.047192949801683426, 0.02450069598853588, -0.015322225168347359, -0.005097086075693369, 0.02028925158083439, -0.014826761558651924, -0.02794417180120945, 0.04310537129640579, -0.039661895483732224, -0.035797275602817535, 0.03718457743525505, -0.03777913376688957, 0.0317096970975399, -0.014975400641560555, -0.006694957613945007, -0.016647592186927795, 0.01771283894777298, -0.002773050218820572, 0.018567515537142754, -0.007332867942750454, 0.008800679817795753, 0.015805302187800407, -0.053212836384773254, 0.013600488193333149, 0.028142357245087624, -0.010169398970901966, -0.00795839074999094, -0.02246929332613945, 0.02417864464223385, -0.025838449597358704, -0.021949056535959244, -0.01621405966579914, -0.0035085047129541636, -0.04221353679895401, 0.00780975166708231, -0.002639894373714924, 0.009729675017297268, 0.02878645993769169, -0.023893753066658974, -0.011086007580161095, -0.011934489943087101, 0.0032019363716244698, 0.04293195903301239, 0.030074667185544968, -0.022580772638320923, 0.016610432416200638, -0.009258984588086605, -0.03453384339809418, -0.0009421558352187276, -0.025144800543785095, 0.010720603168010712, -0.00994644034653902, -0.0065834783017635345, -0.027423935011029243, 0.0286873672157526, -0.01304309070110321, 0.0026693125255405903, -0.019967200234532356, -0.006837403867393732, -0.004251700825989246, -0.05450104549527168, -0.009704901836812496, -0.029777389019727707, 0.004158801399171352, -0.02923237904906273, 0.006103497464209795, -0.03242811933159828, 0.012770585715770721, 0.030644450336694717, -0.004963930230587721, 0.03403837978839874, 0.015631889924407005, -0.03864619508385658, 0.027399161830544472, 0.041693300008773804, 0.006214976776391268, 0.038398463279008865, 0.020264478400349617, -0.015458477661013603, -0.026581645011901855, -0.003008395666256547, 0.027399161830544472, -0.00034217984648421407, -0.0011047299485653639, -0.0367882065474987, -0.011575278826057911, 0.01971946842968464, 0.009711095131933689, 0.0004776582936756313, 0.01490108110010624, 0.004434403032064438, -0.027324842289090157, -0.0046759420074522495, 0.0048152911476790905, 8.931900811148807e-05, -0.03185833618044853, -0.030495811253786087, 0.004217637702822685, 0.014715282246470451, -0.023522155359387398, 0.018047276884317398, -0.005651386454701424, 0.008032710291445255, 0.034806348383426666, 0.03963712230324745, -0.013724354095757008, -0.008026516996324062, -0.012008809484541416, 0.004044224973767996, -0.021973829716444016, -0.020091066136956215, -0.005536810494959354, -0.033369503915309906, 0.00018008956976700574, -0.018034890294075012, 0.01843126304447651, 0.030941728502511978, 0.004164994694292545, 0.02782030589878559, 0.02187473699450493, -0.010937368497252464, -0.01737840101122856, 0.005682353395968676, -0.030000347644090652, -0.014281750656664371, 0.0011302772909402847, -0.01965753547847271, 0.011717724613845348, -0.0032793525606393814, 2.6829571652342565e-05, -0.020450277253985405, -0.00813799723982811, -0.034211792051792145, 0.013489007949829102, -8.685135981068015e-05, 0.05286601185798645, -0.026854149997234344, -0.01684577763080597, -0.0067011509090662, -0.009834961034357548, -0.02764689363539219, -0.024723654612898827, -0.008299022912979126, 0.001749607385136187, -0.035202719271183014, -0.022246334701776505, 0.010677250102162361, -0.018988659605383873, -0.0200167465955019, 0.021168701350688934, 0.028414862230420113, -0.015223132446408272, 0.01116032712161541, 0.006502965465188026, 0.0367882065474987, -0.007512473501265049, -0.04104919359087944, -0.011538119055330753, 0.05950523167848587, 0.0206732377409935, 0.019199231639504433, -0.03066922351717949, 0.007927424274384975, -0.03054535761475563, -0.03309699892997742, -0.010181785561144352, -0.01612735353410244, 0.0024417086970061064, 0.011513345874845982, 0.03718457743525505, 0.0036014041397720575, 0.017700452357530594, -0.030347172170877457, -0.026457779109477997, 0.05217236280441284, 0.044715628027915955, -0.005908408667892218, 0.026408232748508453, 0.003666433971375227, -0.02982693538069725, -0.005063022952526808, -0.02244452014565468, -0.015359384939074516, -0.03143719211220741, -0.025355372577905655, -0.0010621510446071625, 0.009178470820188522, -0.03369155526161194, 0.030198533087968826, -0.009785414673388004, -0.01836932823061943, 0.016164513304829597, 0.01917445845901966, -0.019707081839442253, 0.039092112332582474, 0.001568453386425972, -0.007332867942750454, 0.045904744416475296, 0.05187508463859558, -0.004400339908897877, 0.025466851890087128, -0.0007137779030017555, 0.01849319599568844, 0.007035589311271906, 0.024129098281264305, -0.00167528772726655, -0.0046821353025734425, -0.013117410242557526, 0.0071594552136957645, -0.004214541055262089, -0.043427422642707825, -0.029752615839242935, -0.023522155359387398, -0.03052058443427086, -0.04404675215482712, -0.03096650168299675, -0.018233075737953186, -0.030421491712331772, -0.010163205675780773, -0.008305216208100319, 0.017043963074684143, 0.0006367486785165966, -0.0013594294432550669, -0.0023890656884759665, -0.02638345956802368, -0.03359246253967285, -0.025120027363300323, 0.043873339891433716, -0.014888694509863853, -0.015693822875618935, 0.02893509902060032, -0.026011861860752106, -0.011587665416300297, 0.02393091283738613, 0.0005887506413273513, 0.01737840101122856, 0.004134028218686581, -0.015359384939074516, -0.00012551109830383211, 0.051627352833747864, -0.013476621359586716, 0.0009855089010670781, 0.00462329899892211, 0.036837752908468246, 0.008559141308069229, -0.02327442355453968, -0.003412508638575673, 0.003465151647105813, 0.03128855302929878, -0.014133111573755741, 0.00309355347417295, -0.008311409503221512, -0.021465979516506195, 0.04355128854513168, -0.00655251182615757, 0.027745986357331276, 0.048183877021074295, -0.016015874221920967, 0.03225470706820488, -0.01938503049314022, 0.031164687126874924, -0.00409067515283823, 0.02677983045578003, 0.016437020152807236, -0.027473481371998787, 0.010268491692841053, -0.010386165231466293, 0.027473481371998787, -0.029579203575849533, 0.003846039529889822, -0.029158057644963264, -0.013389915227890015, 0.010522417724132538, 0.002299262909218669, 0.01792341098189354, 0.014479936100542545, 0.045756105333566666, -0.009667742066085339, -0.004626395646482706, 0.022494066506624222, -0.013885379768908024, 0.01672191172838211, 0.02375750057399273, 0.013563328422605991, -0.056235168129205704, 0.06223028153181076, -0.004428209736943245, -0.0349549874663353, 0.007574406452476978, 0.009271371178328991, -0.001239434233866632, 0.0011504056164994836, 0.008230896666646004, -0.0021088188514113426, -0.013142183423042297, -0.02650732547044754, 0.0036850138567388058, -0.012219381518661976, -0.009475749917328358, 0.02115631476044655, -0.005050636362284422, 0.009302337653934956, 0.0017681872704997659, -0.03532658517360687, -0.007549633271992207, -0.042709000408649445, -0.010064112953841686, 0.004155704751610756, 0.029777389019727707, 0.024252964183688164, -0.015396544709801674, 0.02779553271830082, 0.007035589311271906, 0.02354692853987217, 0.012188415043056011, 0.0024509986396878958, 0.018381714820861816, -0.02878645993769169, 0.024723654612898827, 0.010708216577768326, 0.03488066792488098, 0.023076238110661507, 0.026284366846084595, -0.010392358526587486, 0.005704029928892851, -0.009097957983613014, -0.010770149528980255, -0.00011515668302308768, -0.005902215372771025, 0.00813799723982811, 0.00738860759884119, 0.010751569643616676]}>]>]
